In [1]:
!pip install unbabel-comet
from comet import download_model, load_from_checkpoint

In [2]:
!pip -q install transformers accelerate bitsandbytes sentencepiece huggingface_hub

from huggingface_hub import login
login()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.5 MB/s eta 0:00:00


# **Testing Translations on 3 Examples on Original Model**
    - "Qwen/Qwen2.5-3B-Instruct"

In [3]:
# Example set in JSON format
examples = [
    {
        "src": "ठिकाने लगाना",
        "true_meaning": "To eliminate someone.",
        "literal_translation": "To put in place."
    },
    {
        "src": "दिल बाग-बाग होना",
        "true_meaning": "Feeling extremely happy.",
        "literal_translation": "Heart blooming like a garden."
    },
    {
        "src": "जख्म पर नमक छिड़कना",
        "true_meaning": "Adding to someone's pain.",
        "literal_translation": "Sprinkling salt on wounds."
    }
]

In [ ]:
import torch, json
from transformers import AutoTokenizer, AutoModelForCausalLM

# Model
MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE  = torch.bfloat16 if (DEVICE=="cuda" and torch.cuda.is_bf16_supported()) else torch.float16

tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=DTYPE,
    trust_remote_code=True
).eval()

SYSTEM_MSG = "You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone."
def _prompt(text: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user",   "content": f"Hindi: {text}\nEnglish:"}
    ]
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

def _strip_prefix(s: str) -> str:
    s = s.strip()
    low = s.lower()
    if low.startswith("english:"):
        s = s[len("english:"):].strip()
    return s

@torch.no_grad()
def translate_n_best(
    text: str,
    n: int = 3,
    max_new_tokens: int = 96,
    method: str = "sample",
    temperature: float = 0.7,
    top_p: float = 0.95,
    top_k: int = 50,
    num_beams: int = 6,
    diversity_penalty: float = 0.2,
):
    prompt = _prompt(text)
    enc = tok(prompt, return_tensors="pt").to(model.device)
    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        pad_token_id=tok.eos_token_id,
        num_return_sequences=n,
    )

    if method == "sample":
        gen = model.generate(
            **enc,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            **gen_kwargs
        )
    else:
        # beam / diverse beam
        nb = max(num_beams, n)
        gen = model.generate(
            **enc,
            do_sample=False,
            num_beams=nb,
            num_beam_groups=min(n, nb) if diversity_penalty > 0 else 1,
            diversity_penalty=diversity_penalty if diversity_penalty > 0 else 0.0,
            **gen_kwargs
        )

    input_len = enc.input_ids.shape[1]
    outs = []
    for i in range(gen.shape[0]):
        cont = tok.decode(gen[i, input_len:], skip_special_tokens=True)
        outs.append(_strip_prefix(cont))

    # Deduplicate while preserving order, then pad/truncate to n
    seen, unique = set(), []
    for t in outs:
        if t not in seen:
            unique.append(t); seen.add(t)
    unique = (unique + [""] * n)[:n]
    return unique

# Generate top-3 candidates per example
for i, ex in enumerate(examples, 1):
    cand = translate_n_best(
        ex["src"],
        n=3,
        method="sample",
        temperature=0.7,
        top_p=0.95,
        top_k=50,
        num_beams=8,
        diversity_penalty=0.3,
    )
    ex["mt_1"], ex["mt_2"], ex["mt_3"] = cand

    print(f"Example {i}: {ex['src']}")
    print(f"  true_meaning         : {ex['true_meaning']}")
    print(f"  literal_translation  : {ex['literal_translation']}")
    print(f"  mt_1                 : {ex['mt_1']}")
    print(f"  mt_2                 : {ex['mt_2']}")
    print(f"  mt_3                 : {ex['mt_3']}")
    print("-" * 80)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Example 1: ठिकाने लगाना
  true_meaning         : To eliminate someone.
  literal_translation  : To put in place.
  mt_1                 : Locating/Cordoning Off
  mt_2                 : Locating/Areaing
  mt_3                 : Locating or placing
--------------------------------------------------------------------------------
Example 2: दिल बाग-बाग होना
  true_meaning         : Feeling extremely happy.
  literal_translation  : Heart blooming like a garden.
  mt_1                 : The heart craves gardens
  mt_2                 : Heart desires gardens
  mt_3                 : 
--------------------------------------------------------------------------------
Example 3: जख्म पर नमक छिड़कना
  true_meaning         : Adding to someone's pain.
  literal_translation  : Sprinkling salt on wounds.
  mt_1                 : Removing salt from the wound
  mt_2                 : Scraping salt off the bone
  mt_3                 : Scraping salt off the mark
------------------------------------------

# **Evaluate Example Translations Using COMET:**
There are two primary COMET models used for translation evaluation: COMET-QE and COMET-DA.
- COMET-QE evaluates translation quality without requiring a reference, estimating how natural and fluent a translation appears on its own.
- COMET-DA, on the other hand, uses a reference translation to directly compare meaning and fidelity.

For idiomatic or figurative language, COMET-DA is generally more reliable, since COMET-QE often assigns higher scores to literal translations that miss the intended meaning.

The cell below prints the COMET-QE and COMET-DA scores for each of the three machine translations for the above examples. The COMET_QE and COMET-DA scores for the true_meaning and literal_translation from the dataset are also listed. COMET_DA uses the true_meaning from the dataset as the reference (so the COMET_DA score for the true_meaning should be very high or close to 1).

In [ ]:
import math, torch
from comet import download_model, load_from_checkpoint

for ex in examples:
    assert all(k in ex for k in ("src","true_meaning","literal_translation","mt_1","mt_2","mt_3")), \
        "Each example needs src,true_meaning,literal_translation,mt_1..mt_3"

qe_model = load_from_checkpoint(download_model("Unbabel/wmt20-comet-qe-da")).eval()
da_model = load_from_checkpoint(download_model("Unbabel/wmt22-comet-da")).eval()

ALL_LABELS = ("mt_1","mt_2","mt_3","literal_translation","true_meaning")

def _nonempty(s):
    return isinstance(s, str) and len(s.strip()) > 0

def _fmt(x):
    return "nan" if (not isinstance(x,(int,float)) or math.isnan(x)) else f"{x:.3f}"

def comet_qe_scores(ex):
    labels = [lbl for lbl in ALL_LABELS if _nonempty(ex.get(lbl, ""))]
    if not labels:
        return {}
    items = [{"src": ex["src"], "mt": ex[lbl]} for lbl in labels]
    with torch.no_grad():
        scores = qe_model.predict(items, batch_size=16, progress_bar=False)["scores"]
    return {lbl: float(s) for lbl, s in zip(labels, scores)}

def comet_da_scores(ex):
    ref = ex.get("true_meaning", "")
    if not _nonempty(ref):
        return {}  # can't do DA without a non-empty reference
    labels = [lbl for lbl in ALL_LABELS if _nonempty(ex.get(lbl, ""))]
    items = [{"src": ex["src"], "mt": ex[lbl], "ref": ref} for lbl in labels]
    with torch.no_grad():
        scores = da_model.predict(items, batch_size=16, progress_bar=False)["scores"]
    return {lbl: float(s) for lbl, s in zip(labels, scores)}

def best_among_mt(scores_dict):
    # Only consider non-empty mt_k that actually got scored
    mts = [k for k in ("mt_1","mt_2","mt_3") if k in scores_dict]
    if not mts:
        return None, float("nan")
    best = max(mts, key=lambda k: scores_dict[k])
    return best, scores_dict[best]


for ex in examples:
    qe = comet_qe_scores(ex)
    da = comet_da_scores(ex)

    print(f"SRC           : {ex['src']}")
    print(f"REF (meaning) : {ex['true_meaning']}\n")

    # Skipped empty candidates
    skipped = [k for k in ("mt_1","mt_2","mt_3","literal_translation","true_meaning") if not _nonempty(ex.get(k,""))]

    # Print scores for available mt_1..mt_3
    for k in ("mt_1","mt_2","mt_3"):
        if _nonempty(ex.get(k,"")):
            print(f"{k}: {ex[k]}")
            print(f"  -> COMET-QE: {_fmt(qe.get(k, float('nan')))} | COMET-DA (vs ref): {_fmt(da.get(k, float('nan')))}")

    # Literal
    if _nonempty(ex.get("literal_translation","")):
        print("\nDirect translation (literal):", ex["literal_translation"])
        print(f"  -> COMET-QE: {_fmt(qe.get('literal_translation', float('nan')))} | COMET-DA: {_fmt(da.get('literal_translation', float('nan')))}")
    # True meaning as hypothesis
    if _nonempty(ex.get("true_meaning","")):
        print("\nTrue meaning:", ex["true_meaning"])
        print(f"  -> COMET-QE: {_fmt(qe.get('true_meaning', float('nan')))} | COMET-DA: {_fmt(da.get('true_meaning', float('nan')))}")

    # Best among non-empty MT candidates
    qe_best_lbl, qe_best_val = best_among_mt(qe)
    da_best_lbl, da_best_val = best_among_mt(da)

    print("\nBest among MT candidates:")
    if qe_best_lbl:
        print(f"  by COMET-QE: {qe_best_lbl} -> {ex[qe_best_lbl]} (QE={_fmt(qe_best_val)})")
    else:
        print("  by COMET-QE: (none available)")

    if da_best_lbl:
        print(f"  by COMET-DA: {da_best_lbl} -> {ex[da_best_lbl]} (DA={_fmt(da_best_val)})")
    else:
        print("  by COMET-DA: (none available)")
    print("-" * 100)







Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

hparams.yaml:   0%|          | 0.00/479 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

...

SRC           : ठिकाने लगाना
REF (meaning) : To eliminate someone.

mt_1: To locate/place/situate
  -> COMET-QE: -1.019 | COMET-DA (vs ref): 0.529
mt_2: To place/place something
  -> COMET-QE: -1.148 | COMET-DA (vs ref): 0.470
mt_3: Locating
  -> COMET-QE: -0.751 | COMET-DA (vs ref): 0.483

Direct translation (literal): To put in place.
  -> COMET-QE: -0.778 | COMET-DA: 0.474

True meaning: To eliminate someone.
  -> COMET-QE: -0.808 | COMET-DA: 0.979

Best among MT candidates:
  by COMET-QE: mt_3 -> Locating (QE=-0.751)
  by COMET-DA: mt_1 -> To locate/place/situate (DA=0.529)
----------------------------------------------------------------------------------------------------


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytor

SRC           : दिल बाग-बाग होना
REF (meaning) : Feeling extremely happy.

mt_1: Heart yearning for gardens
  -> COMET-QE: -0.362 | COMET-DA (vs ref): 0.512
mt_2: Heart desires gardens
  -> COMET-QE: -0.489 | COMET-DA (vs ref): 0.466

Direct translation (literal): Heart blooming like a garden.
  -> COMET-QE: -0.563 | COMET-DA: 0.457

True meaning: Feeling extremely happy.
  -> COMET-QE: -0.427 | COMET-DA: 0.983

Best among MT candidates:
  by COMET-QE: mt_1 -> Heart yearning for gardens (QE=-0.362)
  by COMET-DA: mt_1 -> Heart yearning for gardens (DA=0.512)
----------------------------------------------------------------------------------------------------


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


SRC           : जख्म पर नमक छिड़कना
REF (meaning) : Adding to someone's pain.

mt_1: Toasting with salt
  -> COMET-QE: -0.508 | COMET-DA (vs ref): 0.393
mt_2: Peeling salt off something
  -> COMET-QE: -0.286 | COMET-DA (vs ref): 0.418

Direct translation (literal): Sprinkling salt on wounds.
  -> COMET-QE: 0.580 | COMET-DA: 0.477

True meaning: Adding to someone's pain.
  -> COMET-QE: -0.770 | COMET-DA: 0.968

Best among MT candidates:
  by COMET-QE: mt_2 -> Peeling salt off something (QE=-0.286)
  by COMET-DA: mt_2 -> Peeling salt off something (DA=0.418)
----------------------------------------------------------------------------------------------------


# **Baseline Evaluation on Whole Dataset (1,000 Examples):**
**Task:** Run model-generated translations (Best of 3) on the dataset and evaluate performance using COMET-QE (reference-free quality estimation) and COMET-DA (reference-based direct assessment).


**Dataset:**
(To-Do: Compute baseline for Chinese Idiom Dataset)

1,000 rows of Hindi idioms with the following columns:

*   src — Hindi idiom (source text)
*   true_meaning — English figurative meaning
*   literal_translation — English literal translation (word-for-word meaning)

**Evaluations:**
Generate three English translations per idiom using the Qwen2.5-3B-Instruct model with varying decoding strategies (greedy + two sampling variants).




1.   Compute COMET-QE scores for each of the three model translations (mt₁, mt₂, mt₃), the literal translation, and the true (reference) meaning.
2.   Select the translation with the highest COMET-QE score as the model’s best output.
3.   Then compute COMET-DA scores to compare this best translation (and optionally the dataset’s provided translation) directly against the reference (true_meaning).




Report:
*   Average COMET-QE for best MT, literal translation, and reference
*   Average COMET-DA for best MT vs. reference
*   Count of cases where best-MT QE > reference QE
*   Count of cases where literal QE > reference QE

Goal: Establish a baseline for model translation quality on idiomatic  translation.


****

In [ ]:
import os, json, math
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# Paths And Config
CSV_PATH = "hindi-english_idioms.csv"  # must have: src, true_meaning, literal_translation
OUT_JSON = "comet_results_3mt.json"
OUT_CSV  = "comet_results_3mt.csv"

MODEL_ID  = "Qwen/Qwen2.5-3B-Instruct"
DEVICE    = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE     = torch.bfloat16 if (DEVICE=="cuda" and torch.cuda.is_bf16_supported()) else torch.float16
MAX_NEW_TOKENS = 96

QE_CKPT = "Unbabel/wmt20-comet-qe-da"

# Load CSV
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

required = {"src", "true_meaning", "literal_translation"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV must have columns {sorted(required)}. Missing: {sorted(missing)}")


# Clean
for col in ["src", "true_meaning", "literal_translation"]:
    df[col] = df[col].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)
print(f"[Info] Loaded {len(df)} rows from {CSV_PATH}")

# Translation model
print(f"[Info] Loading translation model: {MODEL_ID} on {DEVICE}")
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=DTYPE,
    trust_remote_code=True
).eval()

SYSTEM_MSG = "You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone."
def _prompt(text: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user",   "content": f"Hindi: {text}\nEnglish:"}
    ]
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.no_grad()
def translate_once(text: str, *, do_sample: bool, temperature: float = 1.0, top_p: float = 0.9):
    enc = tok(_prompt(text), return_tensors="pt").to(model.device)
    input_len = enc.input_ids.shape[1]
    out = model.generate(
        **enc,
        do_sample=do_sample,
        temperature=temperature if do_sample else None,
        top_p=top_p if do_sample else None,
        max_new_tokens=MAX_NEW_TOKENS,
        pad_token_id=tok.eos_token_id
    )
    cont = tok.decode(out[0, input_len:], skip_special_tokens=True).strip()
    if cont.lower().startswith("english:"):
        cont = cont[len("english:"):].strip()
    return cont

def translate_3_variants(text: str):
    return [
        translate_once(text, do_sample=False),
        translate_once(text, do_sample=True,  temperature=0.7, top_p=0.9),
        translate_once(text, do_sample=True,  temperature=1.0, top_p=0.92)
    ]

# Load COMET (QE + DA)
print("[Info] Loading COMET QE …")
qe_model = load_from_checkpoint(download_model(QE_CKPT)).eval()
print("[Info] Loading COMET DA …")
da_model = load_from_checkpoint(download_model("Unbabel/wmt22-comet-da")).eval()

# Add QE/MT & DA columns
for c in [
    "mt_1","mt_2","mt_3",
    "QE_mt_1","QE_mt_2","QE_mt_3","QE_true","QE_literal","QE_dataset_mt",
    "DA_mt_1","DA_mt_2","DA_mt_3","DA_true","DA_literal",
    "mt_best","QE_mt_best","mt_best_from",
    "mt_best_DA_from","DA_mt_best",
    "literal_gt_ref"
]:
    df[c] = float("nan") if c.startswith(("QE_","DA_")) else ""

# Generate 3 translations and queue QE
print("[Info] Generating 3 translations per row and scoring QE …")
qe_items, qe_locs = [], []

for i, row in df.iterrows():
    src = row["src"]
    mts = translate_3_variants(src)
    df.at[i, "mt_1"], df.at[i, "mt_2"], df.at[i, "mt_3"] = mts

    # Queue QE for MTs
    for j, mt in enumerate(mts, start=1):
        if isinstance(mt, str) and mt.strip():
            qe_items.append({"src": src, "mt": mt})
            qe_locs.append((i, f"QE_mt_{j}"))

    # QE for true and literal (treated as hypotheses)
    if row["true_meaning"].strip():
        qe_items.append({"src": src, "mt": row["true_meaning"]})
        qe_locs.append((i, "QE_true"))
    if row["literal_translation"].strip():
        qe_items.append({"src": src, "mt": row["literal_translation"]})
        qe_locs.append((i, "QE_literal"))


# Batch QE
print(f"[Info] Total QE candidates to score: {len(qe_items)}")
with torch.no_grad():
    qe_scores = qe_model.predict(qe_items, batch_size=16, progress_bar=False)["scores"]
for (i, field), score in zip(qe_locs, qe_scores):
    df.at[i, field] = float(score)

# Pick best MT by QE
def _norm(v):
    try: v = float(v); return v if not math.isnan(v) else -math.inf
    except: return -math.inf

def _best_mt_row(r):
    cands = [("mt_1", r["mt_1"], r["QE_mt_1"]),
             ("mt_2", r["mt_2"], r["QE_mt_2"]),
             ("mt_3", r["mt_3"], r["QE_mt_3"])]
    return max(cands, key=lambda x: _norm(x[2]))

best_cols, best_texts, best_qes = [], [], []
for _, r in df.iterrows():
    col, txt, qe = _best_mt_row(r)
    best_cols.append(col); best_texts.append(txt); best_qes.append(qe)

df["mt_best"] = best_texts
df["QE_mt_best"] = best_qes
df["mt_best_from"] = best_cols

# Literal > reference (QE) flag
qe_true = pd.to_numeric(df["QE_true"], errors="coerce")
qe_lit  = pd.to_numeric(df["QE_literal"], errors="coerce")
df["literal_gt_ref"] = (qe_lit > qe_true)

# COMET-DA
# Score DA for mt_1..3 + literal + true (ref = true_meaning)
da_items, da_locs = [], []
for i, r in df.iterrows():
    ref = r["true_meaning"]
    if not isinstance(ref, str) or not ref.strip():
        continue
    for label in ["mt_1","mt_2","mt_3","literal_translation","true_meaning"]:
        hyp = r[label] if label in r else r.get(label, "")
        if isinstance(hyp, str) and hyp.strip():
            da_items.append({"src": r["src"], "mt": hyp, "ref": ref})
            field = f"DA_{'literal' if label=='literal_translation' else ('true' if label=='true_meaning' else label)}"
            da_locs.append((i, field))

print(f"[Info] Total DA pairs to score: {len(da_items)}")
with torch.no_grad():
    da_scores = da_model.predict(da_items, batch_size=16, progress_bar=False)["scores"]

for (i, field), s in zip(da_locs, da_scores):
    df.at[i, field] = float(s)

# Best-of-3 by DA among MT candidates
da_best_labels, da_best_scores = [], []
for _, r in df.iterrows():
    cands = [("mt_1", r.get("DA_mt_1", float("nan"))),
             ("mt_2", r.get("DA_mt_2", float("nan"))),
             ("mt_3", r.get("DA_mt_3", float("nan")))]
    cands = [(k, (v if isinstance(v,(int,float)) and not math.isnan(v) else -math.inf)) for k, v in cands]
    best_k, best_s = max(cands, key=lambda x: x[1])
    da_best_labels.append(best_k)
    da_best_scores.append(best_s if best_s != -math.inf else float("nan"))

df["mt_best_DA_from"] = da_best_labels
df["DA_mt_best"] = da_best_scores


# Summary helpers
def _nanmean(series):
    vals = pd.to_numeric(series, errors="coerce").dropna().tolist()
    return sum(vals)/len(vals) if vals else float("nan")

# QE Summary
avg_qe_best_mt    = _nanmean(df["QE_mt_best"])
avg_qe_true       = _nanmean(df["QE_true"])
avg_qe_literal    = _nanmean(df["QE_literal"])

num_best_gt_ref = int((pd.to_numeric(df["QE_mt_best"], errors="coerce") >
                       pd.to_numeric(df["QE_true"], errors="coerce")).sum())
num_lit_gt_ref = int(df["literal_gt_ref"].sum())
total_rows = len(df)

print("\n=== SUMMARY (COMET-QE) ===")
print(f"Samples                                        : {total_rows}")
print(f"Average QE (best of 3 MT)                      : {avg_qe_best_mt:.4f}")
print(f"Average QE (literal translation)               : {avg_qe_literal:.4f}")
print(f"Average QE (reference true_meaning)            : {avg_qe_true:.4f}")
print(f"Count where best-MT QE > true meaning QE       : {num_best_gt_ref} / {total_rows}")
print(f"Count where literal QE > true meaning QE       : {num_lit_gt_ref} / {total_rows}")

# DA Summary
avg_da_best_mt = _nanmean(df["DA_mt_best"])            # average DA for DA-best of 3 MTs (per row)
avg_da_true    = _nanmean(df["DA_true"])               # DA(true vs ref) ~ self-reference
avg_da_literal = _nanmean(df["DA_literal"])            # DA(literal vs ref)

print("\n=== SUMMARY (COMET-DA; ref = true_meaning) ===")
print(f"Average DA (best of 3 MT)                      : {avg_da_best_mt:.4f}")
print(f"Average DA (true meaning - Should be close to 1 because the reference is the true meaning)               : {avg_da_true:.4f}")
print(f"Average DA (literal)                    : {avg_da_literal:.4f}")

# Save outputs
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print(f"\nSaved CSV : {OUT_CSV}")
print(f"Saved JSON: {OUT_JSON}")




In [ ]:
!pip install -q transformers accelerate peft unbabel-comet pandas torch --upgrade

from huggingface_hub import login
login()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 136.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 138.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [ ]:
!pip install -U transformers
from huggingface_hub import login
login(new_session=False)

# Trying baseline model Llama-3.2-3B-Instruct

In [ ]:
# === Baseline Evaluation — Llama 3.2-Instruct 3B (QE+DA, best-of-3) ===
import os, json, math, random
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ---------------- Paths and Config ----------------
CSV_PATH = "hindi-english_idioms.csv"   # must have: src, true_meaning, literal_translation
OUT_JSON = "comet_results_3mt_llama3p2_3b.json"
OUT_CSV  = "comet_results_3mt_llama3p2_3b.csv"

MODEL_ID  = "meta-llama/Llama-3.2-3B-Instruct"
DEVICE    = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE     = torch.bfloat16 if (DEVICE=="cuda" and torch.cuda.is_bf16_supported()) else (torch.float16 if DEVICE=="cuda" else torch.float32)
MAX_NEW_TOKENS = 96
SEED = 1234

QE_CKPT = "Unbabel/wmt20-comet-qe-da"
DA_CKPT = "Unbabel/wmt22-comet-da"

# ------------- Repro -------------
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# ------------- Load CSV -------------
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)
required = {"src", "true_meaning", "literal_translation"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV must have columns {sorted(required)}. Missing: {sorted(missing)}")

for col in ["src", "true_meaning", "literal_translation"]:
    df[col] = df[col].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)
print(f"[Info] Loaded {len(df)} rows from {CSV_PATH}")

# ------------- Load Translation Model -------------
print(f"[Info] Loading translation model: {MODEL_ID} on {DEVICE}")
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
if tok.pad_token is None and tok.eos_token is not None:
    tok.pad_token = tok.eos_token
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto" if DEVICE == "cuda" else None,
    torch_dtype=DTYPE,
    trust_remote_code=True
).to(DEVICE).eval()

SYSTEM_MSG = "You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone."
def _prompt(text: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user",   "content": f"Hindi: {text}\nEnglish:"}
    ]
    if getattr(tok, "chat_template", None):
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return f"{SYSTEM_MSG}\nHindi: {text}\nEnglish:"

@torch.no_grad()
def translate_once(text: str, *, do_sample: bool, temperature: float = 1.0, top_p: float = 0.9):
    enc = tok(_prompt(text), return_tensors="pt").to(model.device)
    input_len = enc.input_ids.shape[1]
    out = model.generate(
        **enc,
        do_sample=do_sample,
        temperature=temperature if do_sample else None,
        top_p=top_p if do_sample else None,
        max_new_tokens=MAX_NEW_TOKENS,
        pad_token_id=tok.eos_token_id
    )
    cont = tok.decode(out[0, input_len:], skip_special_tokens=True).strip()
    if cont.lower().startswith("english:"):
        cont = cont[len("english:"):].strip()
    return cont

def translate_3_variants(text: str):
    return [
        translate_once(text, do_sample=False),
        translate_once(text, do_sample=True,  temperature=0.7, top_p=0.9),
        translate_once(text, do_sample=True,  temperature=1.0, top_p=0.92)
    ]

# ------------- Load COMET (QE + DA) -------------
print("[Info] Loading COMET QE …")
qe_model = load_from_checkpoint(download_model(QE_CKPT)).eval()
print("[Info] Loading COMET DA …")
da_model = load_from_checkpoint(download_model(DA_CKPT)).eval()

# ------------- Prepare Columns -------------
for c in [
    "mt_1","mt_2","mt_3",
    "QE_mt_1","QE_mt_2","QE_mt_3","QE_true","QE_literal",
    "DA_mt_1","DA_mt_2","DA_mt_3","DA_true","DA_literal",
    "mt_best","QE_mt_best","mt_best_from",
    "mt_best_DA_from","DA_mt_best",
    "literal_gt_ref"
]:
    df[c] = float("nan") if c.startswith(("QE_","DA_")) else ""

# ------------- Generation + QE -------------
qe_items, qe_locs = [], []
for i, row in df.iterrows():
    src = row["src"]
    mts = translate_3_variants(src)
    df.at[i, "mt_1"], df.at[i, "mt_2"], df.at[i, "mt_3"] = mts

    for j, mt in enumerate(mts, start=1):
        if isinstance(mt, str) and mt.strip():
            qe_items.append({"src": src, "mt": mt})
            qe_locs.append((i, f"QE_mt_{j}"))

    if row["true_meaning"].strip():
        qe_items.append({"src": src, "mt": row["true_meaning"]})
        qe_locs.append((i, "QE_true"))
    if row["literal_translation"].strip():
        qe_items.append({"src": src, "mt": row["literal_translation"]})
        qe_locs.append((i, "QE_literal"))

print(f"[Info] Scoring {len(qe_items)} QE candidates …")
with torch.no_grad():
    qe_scores = qe_model.predict(qe_items, batch_size=16, progress_bar=False)["scores"]
for (i, field), score in zip(qe_locs, qe_scores):
    df.at[i, field] = float(score)

# ------------- Pick QE-best -------------
def _norm(v):
    try: v = float(v); return v if not math.isnan(v) else -math.inf
    except: return -math.inf

best_cols, best_texts, best_qes = [], [], []
for _, r in df.iterrows():
    cands = [("mt_1", r["mt_1"], r["QE_mt_1"]),
             ("mt_2", r["mt_2"], r["QE_mt_2"]),
             ("mt_3", r["mt_3"], r["QE_mt_3"])]
    col, txt, qe = max(cands, key=lambda x: _norm(x[2]))
    best_cols.append(col); best_texts.append(txt); best_qes.append(qe)

df["mt_best"] = best_texts
df["QE_mt_best"] = best_qes
df["mt_best_from"] = best_cols
qe_true = pd.to_numeric(df["QE_true"], errors="coerce")
qe_lit  = pd.to_numeric(df["QE_literal"], errors="coerce")
df["literal_gt_ref"] = (qe_lit > qe_true)

# ------------- COMET-DA (ref = true_meaning) -------------
da_items, da_locs = [], []
for i, r in df.iterrows():
    ref = r["true_meaning"]
    if not ref.strip():
        continue
    for label in ["mt_1","mt_2","mt_3","literal_translation","true_meaning"]:
        hyp = r[label] if label in r else r.get(label, "")
        if isinstance(hyp, str) and hyp.strip():
            da_items.append({"src": r["src"], "mt": hyp, "ref": ref})
            field = f"DA_{'literal' if label=='literal_translation' else ('true' if label=='true_meaning' else label)}"
            da_locs.append((i, field))

print(f"[Info] Scoring {len(da_items)} DA pairs …")
with torch.no_grad():
    da_scores = da_model.predict(da_items, batch_size=16, progress_bar=False)["scores"]
for (i, field), s in zip(da_locs, da_scores):
    df.at[i, field] = float(s)

# ------------- DA-best -------------
da_best_labels, da_best_scores = [], []
for _, r in df.iterrows():
    cands = [("mt_1", r.get("DA_mt_1", float("nan"))),
             ("mt_2", r.get("DA_mt_2", float("nan"))),
             ("mt_3", r.get("DA_mt_3", float("nan")))]
    cands = [(k, v if isinstance(v,(int,float)) and not math.isnan(v) else -math.inf) for k,v in cands]
    best_k, best_s = max(cands, key=lambda x: x[1])
    da_best_labels.append(best_k); da_best_scores.append(best_s if best_s!=-math.inf else float("nan"))
df["mt_best_DA_from"] = da_best_labels
df["DA_mt_best"] = da_best_scores

# ------------- Summaries -------------
def _nanmean(s): vals=pd.to_numeric(s,errors="coerce").dropna(); return vals.mean() if len(vals)>0 else float("nan")

avg_qe_best_mt = _nanmean(df["QE_mt_best"])
avg_qe_true    = _nanmean(df["QE_true"])
avg_qe_literal = _nanmean(df["QE_literal"])
num_best_gt_ref = int((pd.to_numeric(df["QE_mt_best"], errors="coerce") > pd.to_numeric(df["QE_true"], errors="coerce")).sum())
num_lit_gt_ref  = int(df["literal_gt_ref"].sum())

avg_da_best_mt = _nanmean(df["DA_mt_best"])
avg_da_true    = _nanmean(df["DA_true"])
avg_da_literal = _nanmean(df["DA_literal"])

print("\n=== SUMMARY (COMET-QE) — Llama-3.2-3B ===")
print(f"Samples                         : {len(df)}")
print(f"Average QE (best-of-3)           : {avg_qe_best_mt:.4f}")
print(f"Average QE (literal translation) : {avg_qe_literal:.4f}")
print(f"Average QE (true reference)      : {avg_qe_true:.4f}")
print(f"Best-MT QE > true QE             : {num_best_gt_ref} / {len(df)}")
print(f"Literal > reference QE           : {num_lit_gt_ref} / {len(df)}")

print("\n=== SUMMARY (COMET-DA; ref = true_meaning) ===")
print(f"Average DA (best-of-3)           : {avg_da_best_mt:.4f}")
print(f"Average DA (true vs ref)         : {avg_da_true:.4f}")
print(f"Average DA (literal vs ref)      : {avg_da_literal:.4f}")

# ------------- Save -------------
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
print(f"\n Saved CSV : {OUT_CSV}")
print(f"Saved JSON: {OUT_JSON}")



[Info] Loaded 1000 rows from hindi-english_idioms.csv
[Info] Loading translation model: meta-llama/Llama-3.2-3B-Instruct on cuda


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Info] Scoring 5000 QE candidates …


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Info] Scoring 5000 DA pairs …

=== SUMMARY (COMET-QE) — Llama-3.2-3B ===
Samples                         : 1000
Average QE (best-of-3)           : -0.0897
Average QE (literal translation) : -0.2780
Average QE (true reference)      : -0.4171
Best-MT QE > true QE             : 872 / 1000
Literal > reference QE           : 678 / 1000

=== SUMMARY (COMET-DA; ref = true_meaning) ===
Average DA (best-of-3)           : 0.5336
Average DA (true vs ref)         : 0.9742
Average DA (literal vs ref)      : 0.5519

 Saved CSV : comet_results_3mt_llama3p2_3b.csv
Saved JSON: comet_results_3mt_llama3p2_3b.json


In [6]:
# === Baseline Evaluation — Llama 3.2-Instruct 3B (QE+DA, best-of-3) ===
import os, json, math, random, shutil, datetime
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# --- Mount Google Drive and set output folder ---
IN_COLAB = "COLAB_GPU" in os.environ
if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount("/content/drive")
    except Exception as e:
        print("Drive mount skipped:", e)

DRIVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/evals/llama3p2_3b"
os.makedirs(DRIVE_DIR, exist_ok=True)

# ---------------- Paths and Config ----------------
CSV_PATH = "hindi-english_idioms.csv"
OUT_JSON = os.path.join(DRIVE_DIR, "comet_results_3mt_llama3p2_3b.json")
OUT_CSV  = os.path.join(DRIVE_DIR, "comet_results_3mt_llama3p2_3b.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **GRPO Loop Training using COMET QE as the reward function and Qwen/Qwen2.5-3B-Instruct as the baseline model.**


In [ ]:
# ===== GRPO with COMET-QE reward (fixed & batched) — 500 train / 500 test split =====
# - QE-only rewards (Unbabel/wmt20-comet-qe-da)
# - KL sign fixed + adaptive KL
# - per-group advantage standardization
# - batched generation & continuation log-probs
# - grad clipping, TF32, seeding
# - optional: save to Google Drive

import os, json, random, math
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----------------------------- Config -----------------------------
SEED = 1234
CSV_PATH = "hindi-english_idioms.csv"         # columns: src, true_meaning (literal optional)
MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"

# training hyperparams
NUM_EPOCHS = 2
BATCH_SIZE = 8
NUM_SAMPLES_PER_SRC = 4         # candidates per source (training)
LR = 5e-6
WD = 1e-2
GRAD_CLIP_NORM = 1.0
MAX_NEW_TOKENS = 96
GEN_TEMPERATURE = 0.8
GEN_TOP_P = 0.9

# KL control
KL_COEF_INIT = 0.02
TARGET_KL = 0.02

# Saving
USE_DRIVE = True
DRIVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe"  # separate folder for QE runs
LOCAL_DIR = "./qwen2.5-3b-grpo-comet-qe"

# ----------------------------- Repro & perf -----------------------------
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

# ----------------------------- (Optional) Mount Drive -----------------------------
if USE_DRIVE:
    try:
        from google.colab import drive  # type: ignore
        print("→ To save to Drive, run this once in a separate cell if not mounted yet:\nfrom google.colab import drive; drive.mount('/content/drive')")
        os.makedirs(DRIVE_DIR, exist_ok=True)
    except Exception:
        print("Google Colab not detected or drive unavailable; will save locally.")
        USE_DRIVE = False

# ----------------------------- Data -----------------------------
df = pd.read_csv(CSV_PATH)
for col in ["src", "true_meaning"]:
    if col not in df.columns:
        raise ValueError("CSV must contain ['src','true_meaning']")
df["src"] = df["src"].fillna("").astype(str)
df["true_meaning"] = df["true_meaning"].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)

# 500/500 split (or half/half if not exactly 1000)
perm = np.random.RandomState(SEED).permutation(len(df))
mid = len(df) // 2
train_df = df.iloc[perm[:mid]].reset_index(drop=True)
test_df  = df.iloc[perm[mid:]].reset_index(drop=True)  # held out for eval

src_texts = train_df["src"].tolist()
references = train_df["true_meaning"].tolist()  # not used by QE reward, but keep for logging if needed

print(f"[Info] Train size: {len(train_df)} | Test size: {len(test_df)}")

# ----------------------------- Models -----------------------------
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
)
try: model.gradient_checkpointing_enable()
except: pass
model.train()
for p in model.parameters(): p.requires_grad = True

# frozen reference
ref_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
).eval()
for p in ref_model.parameters(): p.requires_grad = False

# reward model (COMET-QE)
qe_model = load_from_checkpoint(download_model("Unbabel/wmt20-comet-qe-da")).eval()
try:
    if DEVICE == "cuda":
        qe_model.model.to("cuda")
except Exception:
    pass

def mdevice(m): return next(m.parameters()).device

# ----------------------------- Generation helpers -----------------------------
# Some builds ignore top_p/temperature unless set via generation_config.
from copy import deepcopy
def gen_texts(model, tok, prompts, num_return_sequences, temperature, top_p, max_new_tokens):
    enc = tok(prompts, return_tensors="pt", padding=True, truncation=True).to(mdevice(model))
    # base config
    gen_cfg = deepcopy(model.generation_config)
    gen_cfg.do_sample = True
    gen_cfg.temperature = float(temperature)
    gen_cfg.top_p = float(top_p)
    out = model.generate(
        **enc,
        generation_config=gen_cfg,
        max_new_tokens=max_new_tokens,
        num_return_sequences=num_return_sequences,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id
    )
    # lengths per prompt (for slicing continuation)
    input_lens = (enc.input_ids != tok.pad_token_id).sum(dim=1)
    return out, input_lens

# ----------------------------- GRPO Trainer (QE) -----------------------------
class GRPOTrainerQE:
    def __init__(self, model, ref_model, tokenizer, qe_model):
        self.model = model
        self.ref_model = ref_model
        self.tok = tokenizer
        self.qe = qe_model
        self.kl_coef = KL_COEF_INIT
        self.target_kl = TARGET_KL

    def _prompt(self, src_text: str) -> str:
        messages = [
            {"role": "system", "content": "You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone."},
            {"role": "user",   "content": f"Hindi: {src_text}\nEnglish:"}
        ]
        return self.tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    @torch.no_grad()
    def generate_batch(self, batch_src, num_samples=NUM_SAMPLES_PER_SRC):
        prompts = [self._prompt(s) for s in batch_src]
        out, input_lens = gen_texts(
            self.model, self.tok, prompts,
            num_return_sequences=num_samples,
            temperature=GEN_TEMPERATURE, top_p=GEN_TOP_P,
            max_new_tokens=MAX_NEW_TOKENS
        )
        all_trans = []
        for i in range(len(batch_src)):
            group = []
            base = i * num_samples
            for k in range(num_samples):
                row = base + k
                cont = out[row][input_lens[i]:]
                txt = self.tok.decode(cont, skip_special_tokens=True).strip()
                if txt.lower().startswith("english:"): txt = txt[len("english:"):].strip()
                group.append(txt)
            all_trans.append(group)
        return all_trans

    @torch.no_grad()
    def comet_qe_rewards(self, batch_src, batch_trans):
        # QE uses (src, mt); higher is better
        items, locs = [], []
        for i, (src, cands) in enumerate(zip(batch_src, batch_trans)):
            for j, t in enumerate(cands):
                if isinstance(t, str) and t.strip():
                    items.append({"src": src, "mt": t})
                    locs.append((i, j))
        scores = [0.0] * len(locs)
        if items:
            sc = self.qe.predict(items, batch_size=32, progress_bar=False)["scores"]
            scores = [float(s) for s in sc]
        rewards = [[] for _ in range(len(batch_src))]
        for (i, j), s in zip(locs, scores):
            while len(rewards[i]) < j: rewards[i].append(0.0)
            rewards[i].append(s)
        for i in range(len(rewards)):
            while len(rewards[i]) < len(batch_trans[i]): rewards[i].append(0.0)
        return rewards

    def _batch_seq_logprob(self, mdl, prompt_ids, cont_ids_list):
        pad_id = self.tok.pad_token_id or self.tok.eos_token_id
        seqs, starts, lens = [], [], []
        for cont in cont_ids_list:
            if cont.numel() == 0:
                cont = torch.tensor([self.tok.eos_token_id], device=prompt_ids.device, dtype=prompt_ids.dtype)
            seqs.append(torch.cat([prompt_ids, cont[:-1]], dim=0))
            starts.append(prompt_ids.size(0) - 1)
            lens.append(cont.size(0))
        inp = torch.nn.utils.rnn.pad_sequence(seqs, batch_first=True, padding_value=pad_id)
        attn = (inp != pad_id).long()
        logits = mdl(input_ids=inp, attention_mask=attn).logits  # [B, T, V]
        outs = []
        for b, (s, L) in enumerate(zip(starts, lens)):
            seg = logits[b, s:s+L, :]
            logp = F.log_softmax(seg, dim=-1)
            cont = cont_ids_list[b]
            lp = logp.gather(-1, cont.unsqueeze(-1)).squeeze(-1).mean()
            outs.append(lp)
        return torch.stack(outs)

    def grpo_loss(self, batch_src, batch_trans, batch_rewards):
        total, groups = 0.0, 0
        kls = []
        for src, cands, rs in zip(batch_src, batch_trans, batch_rewards):
            pairs = [(c, r) for c, r in zip(cands, rs) if isinstance(c, str) and c.strip()]
            if len(pairs) < 2:
                continue

            cands, rs = zip(*pairs)
            r = torch.tensor(rs, device=mdevice(self.model), dtype=torch.float32)
            A = (r - r.mean()) / (r.std() + 1e-6)   # standardized advantages

            prompt = self._prompt(src)
            prompt_ids = tok(prompt, return_tensors="pt").to(mdevice(self.model)).input_ids[0]
            cont_tok = tok(list(cands), return_tensors="pt", add_special_tokens=False, padding=True).to(mdevice(self.model))
            cont_ids = [cont_tok.input_ids[k][cont_tok.attention_mask[k].bool()] for k in range(cont_tok.input_ids.size(0))]

            lp_pi  = self._batch_seq_logprob(self.model,     prompt_ids, cont_ids)
            with torch.no_grad():
                lp_ref = self._batch_seq_logprob(self.ref_model, prompt_ids, cont_ids)

            pg_loss   = -(A.detach() * lp_pi).mean()
            approx_kl = (lp_pi - lp_ref).mean()   # E[log π - log π_ref]
            kls.append(approx_kl.detach())

            loss = pg_loss + self.kl_coef * approx_kl
            total = total + loss
            groups += 1

        # adaptive KL
        if groups > 0 and len(kls) > 0:
            mk = torch.stack(kls).mean().item()
            if   mk > 1.5 * self.target_kl: self.kl_coef *= 1.5
            elif mk < 0.5 * self.target_kl: self.kl_coef /= 1.5

        return total / max(groups, 1)

    def train_step(self, batch_src, optimizer):
        self.model.train()
        translations = self.generate_batch(batch_src, NUM_SAMPLES_PER_SRC)
        rewards = self.comet_qe_rewards(batch_src, translations)
        loss = self.grpo_loss(batch_src, translations, rewards)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), GRAD_CLIP_NORM)
        optimizer.step()
        return loss.item()

# ----------------------------- Train -----------------------------
trainer = GRPOTrainerQE(model, ref_model, tok, qe_model)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)

print("Starting GRPO training with COMET-QE…")
indices = np.arange(len(src_texts))
for epoch in range(NUM_EPOCHS):
    np.random.shuffle(indices)
    total_loss, n_batches = 0.0, 0
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    for i in range(0, len(indices), BATCH_SIZE):
        sel = indices[i:i+BATCH_SIZE]
        batch_src = [src_texts[j] for j in sel]
        loss = trainer.train_step(batch_src, optimizer)
        total_loss += loss; n_batches += 1
        if (i // BATCH_SIZE + 1) % 10 == 0:
            print(f"  Batch {i // BATCH_SIZE + 1}: loss={loss:.4f} | kl_coef={trainer.kl_coef:.4g}")
    print(f"Epoch {epoch+1} avg loss: {total_loss / max(n_batches,1):.4f}")

print("GRPO training with COMET-QE completed!")

# ----------------------------- Save -----------------------------
SAVE_DIR = DRIVE_DIR if USE_DRIVE else LOCAL_DIR
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR, safe_serialization=True)
tok.save_pretrained(SAVE_DIR)
print(f"Saved model & tokenizer to: {SAVE_DIR}")

# (Optional) also save your split for reproducibility
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv",  index=False)
print(f"Splits saved under: {split_dir}")


→ To save to Drive, run this once in a separate cell if not mounted yet:
from google.colab import drive; drive.mount('/content/drive')
[Info] Train size: 500 | Test size: 500


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

...

  Batch 50: loss=-1.4270 | kl_coef=2.528e-22


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.wa

  Batch 60: loss=-2.8890 | kl_coef=4.383e-24


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.wa

Epoch 2 avg loss: -0.6718
GRPO training with COMET-QE completed!
Saved model & tokenizer to: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe
Splits saved under: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe/splits


## Save Models and split csvs to drive folders

In [4]:
!sudo rm -rf /content/drive

# Now mount cleanly
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from pathlib import Path
from transformers import AutoTokenizer

SAVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe"  # or ...-comet-qe
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)

# Save splits
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv",  index=False)

# Save model + tokenizer
model.save_pretrained(SAVE_DIR, safe_serialization=True)
tok.save_pretrained(SAVE_DIR)

# Sanity checks
!ls -lah "{split_dir}"
!ls -lah "{SAVE_DIR}"


total 94K
-rw------- 1 root root 47K Nov  6 04:39 test.csv
-rw------- 1 root root 47K Nov  6 04:39 train.csv
total 5.8G
-rw------- 1 root root  605 Nov  6 04:40 added_tokens.json
-rw------- 1 root root 2.5K Nov  6 04:40 chat_template.jinja
-rw------- 1 root root 1.5K Nov  6 04:39 config.json
-rw------- 1 root root  243 Nov  6 04:39 generation_config.json
-rw------- 1 root root 1.6M Nov  6 04:40 merges.txt
-rw------- 1 root root 4.7G Nov  6 04:40 model-00001-of-00002.safetensors
-rw------- 1 root root 1.2G Nov  6 04:40 model-00002-of-00002.safetensors
-rw------- 1 root root  35K Nov  6 04:40 model.safetensors.index.json
-rw------- 1 root root  613 Nov  6 04:40 special_tokens_map.json
drwx------ 2 root root 4.0K Nov  6 04:39 splits
-rw------- 1 root root 4.6K Nov  6 04:40 tokenizer_config.json
-rw------- 1 root root  11M Nov  6 04:40 tokenizer.json
-rw------- 1 root root 2.7M Nov  6 04:40 vocab.json


# **GRPO Loop Training using COMET DA as the reward function and Qwen/Qwen2.5-3B-Instruct as the baseline model.**

In [ ]:
# ===== GRPO with COMET-DA reward (fixed & batched) — 500 train / 500 test split =====
# - KL sign fixed + adaptive KL
# - per-group advantage standardization
# - batched generation & batched continuation log-probs
# - grad clipping, TF32, seeding
# - optional: save to Google Drive

import os, json, random, math
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----------------------------- Config -----------------------------
SEED = 1234
CSV_PATH = "hindi-english_idioms.csv"         # columns: src, true_meaning (literal optional)
MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"

# training hyperparams
NUM_EPOCHS = 2
BATCH_SIZE = 8
NUM_SAMPLES_PER_SRC = 4         # candidates per source (training)
LR = 5e-6
WD = 1e-2
GRAD_CLIP_NORM = 1.0
MAX_NEW_TOKENS = 96
GEN_TEMPERATURE = 0.8
GEN_TOP_P = 0.9

# KL control
KL_COEF_INIT = 0.02
TARGET_KL = 0.02

# Saving
USE_DRIVE = True
DRIVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da"  # used when USE_DRIVE=True
LOCAL_DIR = "./qwen2.5-3b-grpo-comet-da"

# ----------------------------- Repro & perf -----------------------------
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

# ----------------------------- (Optional) Mount Drive -----------------------------
if USE_DRIVE:
    try:
        from google.colab import drive  # type: ignore
        print("→ To save to Drive, run this once in a separate cell if not mounted yet:\nfrom google.colab import drive; drive.mount('/content/drive')")
        os.makedirs(DRIVE_DIR, exist_ok=True)
    except Exception:
        print("Google Colab not detected or drive unavailable; will save locally.")
        USE_DRIVE = False

# ----------------------------- Data -----------------------------
df = pd.read_csv(CSV_PATH)
for col in ["src", "true_meaning"]:
    if col not in df.columns:
        raise ValueError("CSV must contain ['src','true_meaning']")
df["src"] = df["src"].fillna("").astype(str)
df["true_meaning"] = df["true_meaning"].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)

# 500/500 split (or half/half if not exactly 1000)
perm = np.random.RandomState(SEED).permutation(len(df))
mid = len(df) // 2
train_df = df.iloc[perm[:mid]].reset_index(drop=True)
test_df  = df.iloc[perm[mid:]].reset_index(drop=True)  # not used here, just held out

src_texts = train_df["src"].tolist()
references = train_df["true_meaning"].tolist()

print(f"[Info] Train size: {len(train_df)} | Test size: {len(test_df)}")

# ----------------------------- Models -----------------------------
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
)
try: model.gradient_checkpointing_enable()
except: pass
model.train()
for p in model.parameters(): p.requires_grad = True

# frozen reference
ref_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
).eval()
for p in ref_model.parameters(): p.requires_grad = False

# reward model (COMET-DA)
da_model = load_from_checkpoint(download_model("Unbabel/wmt22-comet-da")).eval()
try:
    if DEVICE == "cuda":
        da_model.model.to("cuda")
except Exception:
    pass

def mdevice(m): return next(m.parameters()).device

# ----------------------------- GRPO Trainer -----------------------------
class GRPOTrainerDA:
    def __init__(self, model, ref_model, tokenizer, da_model):
        self.model = model
        self.ref_model = ref_model
        self.tok = tokenizer
        self.da = da_model
        self.kl_coef = KL_COEF_INIT
        self.target_kl = TARGET_KL

    def _prompt(self, src_text: str) -> str:
        messages = [
            {"role": "system", "content": "You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone."},
            {"role": "user",   "content": f"Hindi: {src_text}\nEnglish:"}
        ]
        return self.tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    @torch.no_grad()
    def generate_batch(self, batch_src, num_samples=NUM_SAMPLES_PER_SRC):
        prompts = [self._prompt(s) for s in batch_src]
        enc = self.tok(prompts, return_tensors="pt", padding=True, truncation=True).to(mdevice(self.model))
        out = self.model.generate(
            **enc,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=True,
            temperature=GEN_TEMPERATURE,
            top_p=GEN_TOP_P,
            num_return_sequences=num_samples,
            pad_token_id=self.tok.eos_token_id
        )
        input_lens = (enc.input_ids != self.tok.pad_token_id).sum(dim=1)
        all_trans = []
        for i in range(len(batch_src)):
            group = []
            base = i * num_samples
            for k in range(num_samples):
                row = base + k
                cont = out[row][input_lens[i]:]
                txt = self.tok.decode(cont, skip_special_tokens=True).strip()
                if txt.lower().startswith("english:"): txt = txt[len("english:"):].strip()
                group.append(txt)
            all_trans.append(group)
        return all_trans

    @torch.no_grad()
    def comet_da_rewards(self, batch_src, batch_trans, batch_refs):
        items, locs = [], []
        for i, (src, cands, ref) in enumerate(zip(batch_src, batch_trans, batch_refs)):
            for j, t in enumerate(cands):
                if isinstance(t, str) and t.strip():
                    items.append({"src": src, "mt": t, "ref": ref})
                    locs.append((i, j))
        scores = [0.0] * len(locs)
        if items:
            sc = self.da.predict(items, batch_size=32, progress_bar=False)["scores"]
            scores = [float(s) for s in sc]
        rewards = [[] for _ in range(len(batch_src))]
        for (i, j), s in zip(locs, scores):
            while len(rewards[i]) < j: rewards[i].append(0.0)
            rewards[i].append(s)
        for i in range(len(rewards)):
            while len(rewards[i]) < len(batch_trans[i]): rewards[i].append(0.0)
        return rewards

    def _batch_seq_logprob(self, mdl, prompt_ids, cont_ids_list):
        pad_id = self.tok.pad_token_id or self.tok.eos_token_id
        seqs, starts, lens = [], [], []
        for cont in cont_ids_list:
            if cont.numel() == 0:
                cont = torch.tensor([self.tok.eos_token_id], device=prompt_ids.device, dtype=prompt_ids.dtype)
            seqs.append(torch.cat([prompt_ids, cont[:-1]], dim=0))
            starts.append(prompt_ids.size(0) - 1)
            lens.append(cont.size(0))
        inp = torch.nn.utils.rnn.pad_sequence(seqs, batch_first=True, padding_value=pad_id)
        attn = (inp != pad_id).long()
        logits = mdl(input_ids=inp, attention_mask=attn).logits  # [B, T, V]
        outs = []
        for b, (s, L) in enumerate(zip(starts, lens)):
            seg = logits[b, s:s+L, :]
            logp = F.log_softmax(seg, dim=-1)
            cont = cont_ids_list[b]
            lp = logp.gather(-1, cont.unsqueeze(-1)).squeeze(-1).mean()
            outs.append(lp)
        return torch.stack(outs)

    def grpo_loss(self, batch_src, batch_trans, batch_rewards):
        total, groups = 0.0, 0
        kls = []
        for src, cands, rs in zip(batch_src, batch_trans, batch_rewards):
            pairs = [(c, r) for c, r in zip(cands, rs) if isinstance(c, str) and c.strip()]
            if len(pairs) < 2:
                continue

            cands, rs = zip(*pairs)
            r = torch.tensor(rs, device=mdevice(self.model), dtype=torch.float32)
            A = (r - r.mean()) / (r.std() + 1e-6)   # standardized advantages

            prompt = self._prompt(src)
            prompt_ids = tok(prompt, return_tensors="pt").to(mdevice(self.model)).input_ids[0]
            cont_tok = tok(list(cands), return_tensors="pt", add_special_tokens=False, padding=True).to(mdevice(self.model))
            cont_ids = [cont_tok.input_ids[k][cont_tok.attention_mask[k].bool()] for k in range(cont_tok.input_ids.size(0))]

            lp_pi  = self._batch_seq_logprob(self.model,     prompt_ids, cont_ids)
            with torch.no_grad():
                lp_ref = self._batch_seq_logprob(self.ref_model, prompt_ids, cont_ids)

            pg_loss   = -(A.detach() * lp_pi).mean()
            approx_kl = (lp_pi - lp_ref).mean()   # correct sign: E[log π - log π_ref]
            kls.append(approx_kl.detach())

            loss = pg_loss + self.kl_coef * approx_kl
            total = total + loss
            groups += 1

        # simple adaptive KL towards target
        if groups > 0 and len(kls) > 0:
            mk = torch.stack(kls).mean().item()
            if   mk > 1.5 * self.target_kl: self.kl_coef *= 1.5
            elif mk < 0.5 * self.target_kl: self.kl_coef /= 1.5

        return total / max(groups, 1)

    def train_step(self, batch_src, batch_refs, optimizer):
        self.model.train()
        translations = self.generate_batch(batch_src, NUM_SAMPLES_PER_SRC)
        rewards = self.comet_da_rewards(batch_src, translations, batch_refs)
        loss = self.grpo_loss(batch_src, translations, rewards)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), GRAD_CLIP_NORM)
        optimizer.step()
        return loss.item()

# ----------------------------- Train -----------------------------
trainer = GRPOTrainerDA(model, ref_model, tok, da_model)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)

print("Starting GRPO training with COMET-DA…")
indices = np.arange(len(src_texts))
for epoch in range(NUM_EPOCHS):
    np.random.shuffle(indices)
    total_loss, n_batches = 0.0, 0
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    for i in range(0, len(indices), BATCH_SIZE):
        sel = indices[i:i+BATCH_SIZE]
        batch_src = [src_texts[j] for j in sel]
        batch_ref = [references[j] for j in sel]
        loss = trainer.train_step(batch_src, batch_ref, optimizer)
        total_loss += loss; n_batches += 1
        if (i // BATCH_SIZE + 1) % 10 == 0:
            print(f"  Batch {i // BATCH_SIZE + 1}: loss={loss:.4f} | kl_coef={trainer.kl_coef:.4g}")
    print(f"Epoch {epoch+1} avg loss: {total_loss / max(n_batches,1):.4f}")

print("GRPO training with COMET-DA completed!")

# ----------------------------- Save -----------------------------
SAVE_DIR = DRIVE_DIR if USE_DRIVE else LOCAL_DIR
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tok.save_pretrained(SAVE_DIR)
print(f"Saved model & tokenizer to: {SAVE_DIR}")

# (Optional) also save your split for reproducibility
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv",  index=False)
print(f"Splits saved under: {split_dir}")



→ To save to Drive, run this once in a separate cell if not mounted yet:
from google.colab import drive; drive.mount('/content/drive')
[Info] Train size: 500 | Test size: 500


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

  Batch 50: loss=-2.3543 | kl_coef=8.405e-19


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable Li

  Batch 60: loss=-2.4551 | kl_coef=1.458e-20


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable Li

Epoch 2 avg loss: -1.4276
GRPO training with COMET-DA completed!
Saved model & tokenizer to: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da
Splits saved under: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da/splits


# Save Models and split csvs to drive folders

In [ ]:
!sudo rm -rf /content/drive

# Now mount cleanly
from google.colab import drive
drive.mount('/content/drive')

# 2) Choose where to save on Drive
SAVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da"
from pathlib import Path
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)

# 3) Save the splits under .../splits/
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv",  index=False)

# 4) (Optional) sanity check
!ls -lah "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da/splits"

rm: cannot remove '/content/drive/.shortcut-targets-by-id': Operation canceled
rm: cannot remove '/content/drive/Shareddrives/Melanoma Dataset': Operation not supported
rm: cannot remove '/content/drive/MyDrive': Operation canceled
rm: cannot remove '/content/drive/.Trash-0': Directory not empty
rm: cannot remove '/content/drive/.Encrypted/.shortcut-targets-by-id': Operation canceled
rm: cannot remove '/content/drive/.Encrypted/Shareddrives/Melanoma Dataset': Operation not supported
rm: cannot remove '/content/drive/.Encrypted/MyDrive': Operation canceled
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 94K
-rw------- 1 root root 47K Nov  6 21:07 test.csv
-rw------- 1 root root 47K Nov  6 21:07 train.csv


In [ ]:
# Save the current in-memory model + tokenizer to Drive
from transformers import AutoTokenizer
import os
SAVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da"
os.makedirs(SAVE_DIR, exist_ok=True)

# weights (use safetensors if possible)
model.save_pretrained(SAVE_DIR, safe_serialization=True)

# tokenizer from base repo
tok = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct", trust_remote_code=True, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
tok.save_pretrained(SAVE_DIR)

!ls -lah "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da"

total 5.8G
-rw------- 1 root root  605 Nov  6 04:51 added_tokens.json
-rw------- 1 root root 2.5K Nov  6 04:51 chat_template.jinja
-rw------- 1 root root 1.5K Nov  6 04:51 config.json
-rw------- 1 root root  243 Nov  6 04:51 generation_config.json
-rw------- 1 root root 1.6M Nov  6 04:51 merges.txt
-rw------- 1 root root 4.7G Nov  6 04:51 model-00001-of-00002.safetensors
-rw------- 1 root root 1.2G Nov  6 04:51 model-00002-of-00002.safetensors
-rw------- 1 root root  35K Nov  6 04:51 model.safetensors.index.json
-rw------- 1 root root  613 Nov  6 04:51 special_tokens_map.json
-rw------- 1 root root 4.6K Nov  6 04:51 tokenizer_config.json
-rw------- 1 root root  11M Nov  6 04:51 tokenizer.json
-rw------- 1 root root 2.7M Nov  6 04:51 vocab.json


#  Trying another hybrid reward model (since there isn't much improvement with COMET-QE and COMET-DA)

In [ ]:
import os, json, random, math, re
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer as HFTokenizer, AutoModelForSequenceClassification
from comet import download_model, load_from_checkpoint

# ----------------------------- Config -----------------------------
SEED = 1234
CSV_PATH = "hindi-english_idioms.csv"         # columns: src, true_meaning, (optional) literal_translation
MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"

# training hyperparams
NUM_EPOCHS = 2
BATCH_SIZE = 8
NUM_SAMPLES_PER_SRC = 4         # candidates per source (training)
LR = 5e-6
WD = 1e-2
GRAD_CLIP_NORM = 1.0
MAX_NEW_TOKENS = 96
GEN_TEMPERATURE = 0.8
GEN_TOP_P = 0.9
REPETITION_PENALTY = 1.05

# KL control
KL_COEF_INIT = 0.02
TARGET_KL = 0.02

# Hybrid reward weights
ALPHA = 0.35  # QE
BETA  = 0.35  # DA
GAMMA = 0.30  # NLI entailment prob
LAMBDA = 0.40 # literal-overlap penalty

# Saving
USE_DRIVE = True
DRIVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid"  # new dir name for hybrid
LOCAL_DIR = "./qwen2.5-3b-grpo-hybrid"

# ----------------------------- Repro & perf -----------------------------
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

# ----------------------------- (Optional) Mount Drive -----------------------------
if USE_DRIVE:
    try:
        from google.colab import drive  # type: ignore
        print("→ To save to Drive, run this once in a separate cell if not mounted yet:\nfrom google.colab import drive; drive.mount('/content/drive')")
        os.makedirs(DRIVE_DIR, exist_ok=True)
    except Exception:
        print("Google Colab not detected or drive unavailable; will save locally.")
        USE_DRIVE = False

# ----------------------------- Data -----------------------------
df = pd.read_csv(CSV_PATH)
for col in ["src", "true_meaning"]:
    if col not in df.columns:
        raise ValueError("CSV must contain ['src','true_meaning']")
if "literal_translation" not in df.columns:
    df["literal_translation"] = ""
for k in ["src","true_meaning","literal_translation"]:
    df[k] = df[k].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)

# ----------------------------- 800/200 split (fallback to 80/20 if <1000) -----------------------------
perm = np.random.RandomState(SEED).permutation(len(df))

if len(df) >= 1000:
    sel = perm[:1000]
    train_idx = sel[:800]
    test_idx  = sel[800:1000]
else:
    cutoff    = max(1, int(round(0.8 * len(df))))
    train_idx = perm[:cutoff]
    test_idx  = perm[cutoff:]

train_df = df.iloc[train_idx].reset_index(drop=True)
test_df  = df.iloc[test_idx].reset_index(drop=True)

src_texts  = train_df["src"].tolist()
references = train_df["true_meaning"].tolist()
literals   = train_df["literal_translation"].tolist()

print(f"[Info] Train size: {len(train_df)} | Test size: {len(test_df)}")

# ----------------------------- Models -----------------------------
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
if tok.pad_token_id is None: tok.pad_token = tok.eos_token
tok.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
)
try: model.gradient_checkpointing_enable()
except: pass
model.train()
for p in model.parameters(): p.requires_grad = True

# frozen reference
ref_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
).eval()
for p in ref_model.parameters(): p.requires_grad = False

# reward models
qe_model = load_from_checkpoint(download_model("Unbabel/wmt20-comet-qe-da")).eval()
da_model = load_from_checkpoint(download_model("Unbabel/wmt22-comet-da")).eval()
try:
    if DEVICE == "cuda":
        qe_model.model.to("cuda")
        da_model.model.to("cuda")
except Exception:
    pass

# NLI (entailment: MT ⇒ true_meaning)
NLI_CKPT = "roberta-large-mnli"
nli_tok = HFTokenizer.from_pretrained(NLI_CKPT)
nli_mod = AutoModelForSequenceClassification.from_pretrained(NLI_CKPT).to(DEVICE).eval()
ENTAILMENT_ID = 2

def mdevice(m): return next(m.parameters()).device

# ----------------------------- Simple Prompt -----------------------------
SYSTEM_MSG = (
    "Translate Hindi to natural English. If the text is figurative or idiomatic, convey the intended TRUE meaning. "
    "Output ONE English line only."
)

def build_prompt(src_text: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user",   "content": f"Hindi: {src_text}\nEnglish:"}
    ]
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# ----------------------------- Scoring utilities -----------------------------
@torch.no_grad()
def nli_entail_prob(premise: str, hypothesis: str) -> float:
    if not premise or not hypothesis: return 0.0
    enc = nli_tok(premise, hypothesis, return_tensors="pt", truncation=True, padding=True).to(DEVICE)
    logits = nli_mod(**enc).logits.float()
    return float(logits.softmax(dim=-1)[0, ENTAILMENT_ID].item())

def literal_overlap_penalty(mt: str, lit: str) -> float:
    # token Jaccard in [0,1]: higher = more literal (worse)
    if not lit or not mt: return 0.0
    A = set(re.findall(r"[a-z]+", mt.lower()))
    B = set(re.findall(r"[a-z]+", lit.lower()))
    if not A or not B: return 0.0
    return len(A & B) / max(1, len(A | B))

class HybridReward:
    def __init__(self, qe_model, da_model, alpha=ALPHA, beta=BETA, gamma=GAMMA, lam=LAMBDA):
        self.qe = qe_model
        self.da = da_model
        self.alpha = alpha; self.beta = beta; self.gamma = gamma; self.lam = lam

    @torch.no_grad()
    def score(self, src_list, cand_lists, ref_list, lit_list):
        # COMET-QE (src, mt)
        qe_items, qe_map = [], []
        for i,(src,cands) in enumerate(zip(src_list, cand_lists)):
            for j,mt in enumerate(cands):
                if isinstance(mt,str) and mt.strip():
                    qe_items.append({"src":src, "mt":mt}); qe_map.append((i,j))
        qe_scores = [0.0]*len(qe_map)
        if qe_items:
            qe_scores = [float(s) for s in self.qe.predict(qe_items, batch_size=32, progress_bar=False)["scores"]]

        # COMET-DA (src, mt, ref)
        da_items, da_map = [], []
        for i,(src,cands,ref) in enumerate(zip(src_list, cand_lists, ref_list)):
            if not (isinstance(ref,str) and ref.strip()): continue
            for j,mt in enumerate(cands):
                if isinstance(mt,str) and mt.strip():
                    da_items.append({"src":src, "mt":mt, "ref":ref}); da_map.append((i,j))
        da_scores = [0.0]*len(da_map)
        if da_items:
            da_scores = [float(s) for s in self.da.predict(da_items, batch_size=32, progress_bar=False)["scores"]]

        # reassemble and add NLI + literal penalty
        rewards = [[0.0 for _ in cands] for cands in cand_lists]
        for (i,j),s in zip(qe_map, qe_scores): rewards[i][j] += self.alpha * s
        for (i,j),s in zip(da_map, da_scores): rewards[i][j] += self.beta  * s

        for i,(cands,ref,lit) in enumerate(zip(cand_lists, ref_list, lit_list)):
            for j,mt in enumerate(cands):
                if not (isinstance(mt,str) and mt.strip()): continue
                ent = nli_entail_prob(mt, ref) if (isinstance(ref,str) and ref.strip()) else 0.0
                penalties = literal_overlap_penalty(mt, lit) if (isinstance(lit,str) and lit.strip()) else 0.0
                rewards[i][j] += self.gamma * ent
                rewards[i][j] -= self.lam   * penalties
        return rewards

hybrid = HybridReward(qe_model, da_model)

# ----------------------------- GRPO Trainer -----------------------------
class GRPOTrainerHybrid:
    def __init__(self, model, ref_model, tokenizer):
        self.model = model
        self.ref_model = ref_model
        self.tok = tokenizer
        self.kl_coef = KL_COEF_INIT
        self.target_kl = TARGET_KL

    def _prompt(self, src_text: str) -> str:
        return build_prompt(src_text)

    @torch.no_grad()
    def generate_batch(self, batch_src, num_samples=NUM_SAMPLES_PER_SRC):
        prompts = [self._prompt(s) for s in batch_src]
        enc = self.tok(prompts, return_tensors="pt", padding=True, truncation=True).to(mdevice(self.model))
        out = self.model.generate(
            **enc,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=True,
            temperature=GEN_TEMPERATURE,
            top_p=GEN_TOP_P,
            repetition_penalty=REPETITION_PENALTY,
            num_return_sequences=num_samples,
            pad_token_id=self.tok.pad_token_id or self.tok.eos_token_id,
            eos_token_id=self.tok.eos_token_id,
        )
        input_lens = (enc.input_ids != self.tok.pad_token_id).sum(dim=1)
        all_trans = []
        for i in range(len(batch_src)):
            group, base = [], i * num_samples
            for k in range(num_samples):
                row = base + k
                cont = out[row][input_lens[i]:]
                txt = self.tok.decode(cont, skip_special_tokens=True).strip()
                for p in ("English:","english:","Translation:","translation:"):
                    if txt.startswith(p): txt = txt[len(p):].strip()
                txt = " ".join(txt.split())[:140]
                group.append(txt)
            all_trans.append(group)
        return all_trans

    def _batch_seq_logprob(self, mdl, prompt_ids, cont_ids_list):
        pad_id = self.tok.pad_token_id or self.tok.eos_token_id
        seqs, starts, lens = [], [], []
        for cont in cont_ids_list:
            if cont.numel() == 0:
                cont = torch.tensor([self.tok.eos_token_id], device=prompt_ids.device, dtype=prompt_ids.dtype)
            seqs.append(torch.cat([prompt_ids, cont[:-1]], dim=0))
            starts.append(prompt_ids.size(0) - 1)
            lens.append(cont.size(0))
        inp = torch.nn.utils.rnn.pad_sequence(seqs, batch_first=True, padding_value=pad_id)
        attn = (inp != pad_id).long()
        logits = mdl(input_ids=inp, attention_mask=attn).logits
        outs = []
        for b,(s,L) in enumerate(zip(starts, lens)):
            seg = logits[b, s:s+L, :]
            logp = F.log_softmax(seg, dim=-1)
            cont = cont_ids_list[b]
            lp = logp.gather(-1, cont.unsqueeze(-1)).squeeze(-1).mean()
            outs.append(lp)
        return torch.stack(outs)

    def grpo_loss(self, batch_src, batch_trans, batch_rewards):
        total, groups = 0.0, 0
        kls = []
        for src, cands, rs in zip(batch_src, batch_trans, batch_rewards):
            pairs = [(c, r) for c, r in zip(cands, rs) if isinstance(c, str) and c.strip()]
            if len(pairs) < 2:  # need >=2 for group ranking/advantages
                continue

            cands, rs = zip(*pairs)
            r = torch.tensor(rs, device=mdevice(self.model), dtype=torch.float32)
            A = (r - r.mean()) / (r.std() + 1e-6)   # standardized advantages

            prompt = self._prompt(src)
            prompt_ids = tok(prompt, return_tensors="pt").to(mdevice(self.model)).input_ids[0]
            cont_tok = tok(list(cands), return_tensors="pt", add_special_tokens=False, padding=True).to(mdevice(self.model))
            cont_ids = [cont_tok.input_ids[k][cont_tok.attention_mask[k].bool()] for k in range(cont_tok.input_ids.size(0))]

            lp_pi  = self._batch_seq_logprob(self.model,     prompt_ids, cont_ids)
            with torch.no_grad():
                lp_ref = self._batch_seq_logprob(self.ref_model, prompt_ids, cont_ids)

            pg_loss   = -(A.detach() * lp_pi).mean()
            approx_kl = (lp_pi - lp_ref).mean()   # E[log π - log π_ref]
            kls.append(approx_kl.detach())

            loss = pg_loss + self.kl_coef * approx_kl
            total = total + loss
            groups += 1

        if groups > 0 and len(kls) > 0:
            mk = torch.stack(kls).mean().item()
            if   mk > 1.5 * self.target_kl: self.kl_coef *= 1.5
            elif mk < 0.5 * self.target_kl: self.kl_coef /= 1.5

        return total / max(groups, 1)

    def train_step(self, batch_src, batch_refs, batch_lits, optimizer):
        self.model.train()
        translations = self.generate_batch(batch_src, NUM_SAMPLES_PER_SRC)
        rewards = hybrid.score(batch_src, translations, batch_refs, batch_lits)
        loss = self.grpo_loss(batch_src, translations, rewards)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), GRAD_CLIP_NORM)
        optimizer.step()
        return loss.item()

# ----------------------------- Train -----------------------------
trainer = GRPOTrainerHybrid(model, ref_model, tok)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)

print("Starting GRPO training with HYBRID reward…")
indices = np.arange(len(src_texts))
for epoch in range(NUM_EPOCHS):
    np.random.shuffle(indices)
    total_loss, n_batches = 0.0, 0
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    for i in range(0, len(indices), BATCH_SIZE):
        sel = indices[i:i+BATCH_SIZE]
        batch_src = [src_texts[j]  for j in sel]
        batch_ref = [references[j] for j in sel]
        batch_lit = [literals[j]   for j in sel]
        loss = trainer.train_step(batch_src, batch_ref, batch_lit, optimizer)
        total_loss += loss; n_batches += 1
        if (i // BATCH_SIZE + 1) % 10 == 0:
            print(f"  Batch {i // BATCH_SIZE + 1}: loss={loss:.4f} | kl_coef={trainer.kl_coef:.4g}")
    print(f"Epoch {epoch+1} avg loss: {total_loss / max(n_batches,1):.4f}")

print("GRPO training with HYBRID reward completed!")

# ----------------------------- Save -----------------------------
SAVE_DIR = DRIVE_DIR if USE_DRIVE else LOCAL_DIR
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tok.save_pretrained(SAVE_DIR)
print(f"Saved model & tokenizer to: {SAVE_DIR}")

# (Optional) also save your split for reproducibility
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv",  index=False)
print(f"Splits saved under: {split_dir}")



→ To save to Drive, run this once in a separate cell if not mounted yet:
from google.colab import drive; drive.mount('/content/drive')
[Info] Train size: 800 | Test size: 200


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt20-comet-qe-da/snapshots/2e7ffc84fb67d99cf92506611766463bb9230cfb/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


...

  Batch 80: loss=-4.1786 | kl_coef=4.023e-34


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:p

  Batch 90: loss=-4.1576 | kl_coef=6.977e-36


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:p

  Batch 100: loss=-3.1706 | kl_coef=1.21e-37
Epoch 2 avg loss: -2.8259
GRPO training with HYBRID reward completed!
Saved model & tokenizer to: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid
Splits saved under: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid/splits


In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 2) Choose a save folder (match whatever you used in training)
SAVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid"
from pathlib import Path
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)

# 3) Save splits
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv",  index=False)

# 4) Sanity check (Colab shell magic)
!ls -lah "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid/splits"

Mounted at /content/drive
total 94K
-rw------- 1 root root 19K Nov  5 23:38 test.csv
-rw------- 1 root root 75K Nov  5 23:38 train.csv


In [ ]:
# Save fine-tuned model + tokenizer to Google Drive
from transformers import AutoTokenizer
import os

# Use the same directory as your training script
SAVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid"
os.makedirs(SAVE_DIR, exist_ok=True)

# Save model weights (prefer safetensors if supported)
model.save_pretrained(SAVE_DIR, safe_serialization=True)

# Save tokenizer from base repo (ensures pad_token is set)
tok = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct", trust_remote_code=True, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
tok.save_pretrained(SAVE_DIR)

# Confirm that files are saved
!ls -lah "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid"


total 5.8G
-rw------- 1 root root  605 Nov  5 23:38 added_tokens.json
-rw------- 1 root root 2.5K Nov  5 23:38 chat_template.jinja
-rw------- 1 root root 1.5K Nov  5 23:38 config.json
-rw------- 1 root root  243 Nov  5 23:38 generation_config.json
-rw------- 1 root root 1.6M Nov  5 23:38 merges.txt
-rw------- 1 root root 4.7G Nov  5 23:38 model-00001-of-00002.safetensors
-rw------- 1 root root 1.2G Nov  5 23:38 model-00002-of-00002.safetensors
-rw------- 1 root root  35K Nov  5 23:38 model.safetensors.index.json
-rw------- 1 root root  613 Nov  5 23:38 special_tokens_map.json
drwx------ 2 root root 4.0K Nov  5 23:38 splits
-rw------- 1 root root 4.6K Nov  5 23:38 tokenizer_config.json
-rw------- 1 root root  11M Nov  5 23:38 tokenizer.json
-rw------- 1 root root 2.7M Nov  5 23:38 vocab.json


# Another Approach: Fine-Tuning Strategy: Qwen2.5-3B-Instruct with LoRA SFT And GRPO Training (COMET-DA as the reward)

In [ ]:
import os, sys, subprocess, random, math, json, time
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def pip_install(pkgs): subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)
pip_install(["-U", "transformers>=4.44.0", "accelerate>=0.33.0", "peft>=0.11.1", "unbabel-comet", "datasets"])

# Mount Drive
IN_COLAB = "COLAB_GPU" in os.environ
if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount("/content/drive")
    except Exception as e:
        print("Drive mount skipped:", e)

CSV_PATH   = "hindi-english_idioms.csv"  # CSV columns: src,true_meaning (literal_translation optional)
SAVE_DIR   = "/content/drive/MyDrive/newfolder/qwen2.5-3b-grpo-comet-da-fast_800-200"
MODEL_ID   = "Qwen/Qwen2.5-3B-Instruct"
SEED = 1234

# SFT (warm-start; OOM-safe)
SFT_EPOCHS = 1
SFT_LR     = 1e-4
SFT_WD     = 0.01
SFT_BATCH  = 4
SFT_MAXLEN = 192
SFT_GRAD_ACCUM = 8  # effective batch 32

# GRPO
NUM_EPOCHS = 2
BATCH_SIZE = 8
GRAD_ACCUM = 2
NUM_SAMPLES_PER_SRC = 2
LR = 5e-6
WD = 1e-2
GRAD_CLIP_NORM = 1.0

# Decoding
MAX_NEW_TOKENS = 48
GEN_TEMPERATURE = 0.7
GEN_TOP_P = 0.9

# KL control
TARGET_KL = 0.01
KL_COEF_INIT = 0.02

# Mixed reward weights
LITERAL_W = 0.4
LEN_W     = 0.1
TARGET_LEN = 18  # tokens

# Misc
USE_BF16 = True  # prefer bf16 on A100

import numpy as np, torch
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported() and USE_BF16:
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32
print("Device:", DEVICE, "| DType:", DTYPE)

# ----------------------------- Load Data + 800/200 split -----------------------------
import pandas as pd
from pathlib import Path
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)
for col in ["src", "true_meaning"]:
    assert col in df.columns, "CSV must contain columns: src,true_meaning"
df["src"] = df["src"].fillna("").astype(str)
df["true_meaning"] = df["true_meaning"].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)

N = len(df)
assert N >= 200, "Need at least 200 rows for 800/200 or proportional split."
train_size = min(800, max(1, N - 200))
test_size  = N - train_size
perm = np.random.RandomState(SEED).permutation(N)
train_df = df.iloc[perm[:train_size]].reset_index(drop=True)
test_df  = df.iloc[perm[train_size:]].reset_index(drop=True)

src_texts = train_df["src"].tolist()
references = train_df["true_meaning"].tolist()
print(f"Train: {len(train_df)} | Test: {len(test_df)}")
print(train_df.head(2))

# Tokenizer + bf16/fp16 LoRA model (no quant)
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
if tok.pad_token_id is None:
    tok.pad_token_id = tok.eos_token_id

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", dtype=DTYPE, trust_remote_code=True
)
try: base_model.gradient_checkpointing_enable()
except: pass

lora_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)
model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()

# 1-epoch LoRA SFT Warm-Start (streamed mini-batches)
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

def sft_prompt(src, tgt):
    return f"Translate the Hindi idiom to natural English (figurative meaning only).\nHindi: {src}\nEnglish: {tgt}"

class SFTDataset(Dataset):
    def __init__(self, srcs, tgts, tokenizer, max_len):
        self.srcs, self.tgts, self.tok, self.max_len = srcs, tgts, tokenizer, max_len
    def __len__(self): return len(self.srcs)
    def __getitem__(self, i):
        text = sft_prompt(self.srcs[i], self.tgts[i])
        enc = self.tok(text, return_tensors="pt", padding=False, truncation=True, max_length=self.max_len)
        return {k: v.squeeze(0) for k, v in enc.items()}

def collate(batch):
    pad = tok.pad_token_id
    ids = [b["input_ids"] for b in batch]
    attn = [b["attention_mask"] for b in batch]
    ids = torch.nn.utils.rnn.pad_sequence(ids, batch_first=True, padding_value=pad)
    attn = torch.nn.utils.rnn.pad_sequence(attn, batch_first=True, padding_value=0)
    return {"input_ids": ids, "attention_mask": attn, "labels": ids.clone()}

sft_ds = SFTDataset(src_texts, references, tok, SFT_MAXLEN)
sft_dl = DataLoader(sft_ds, batch_size=SFT_BATCH, shuffle=True, collate_fn=collate)

opt = AdamW(model.parameters(), lr=SFT_LR, weight_decay=SFT_WD)
model.train()
global_sft_step = 0
for ep in range(SFT_EPOCHS):
    running = 0.0
    opt.zero_grad(set_to_none=True)
    for step, batch in enumerate(sft_dl, 1):
        batch = {k: v.to(model.device) for k, v in batch.items()}
        out = model(**batch)
        loss = out.loss / SFT_GRAD_ACCUM
        loss.backward()
        running += loss.item()
        if step % SFT_GRAD_ACCUM == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step(); opt.zero_grad(set_to_none=True)
        if step % 50 == 0:
            print(f"SFT epoch {ep+1} step {step}/{len(sft_dl)} — loss(avg_accum): {running:.4f}")
            running = 0.0
    # flush remaining grads
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    opt.step(); opt.zero_grad(set_to_none=True)
    torch.cuda.empty_cache()
print("SFT done.")

# ----------------------------- Load frozen reference model AFTER SFT (saves memory) -----------------------------
ref_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", dtype=DTYPE, trust_remote_code=True
).eval()
for p in ref_model.parameters(): p.requires_grad = False
torch.cuda.empty_cache()

# ----------------------------- COMET-DA + Reward Helpers (cached mixed reward) -----------------------------
import torch.nn.functional as F
from comet import download_model, load_from_checkpoint

print("Loading COMET-DA…")
da_model = load_from_checkpoint(download_model("Unbabel/wmt22-comet-da")).eval()
try:
    if DEVICE == "cuda":
        da_model.model.to("cuda")
except Exception as e:
    print("COMET GPU move warning:", e)

def jaccard_overlap(ids_a, ids_b):
    A, B = set(ids_a), set(ids_b)
    return len(A & B) / max(1, len(A | B))

reward_cache = {}  # (hash(src), hash(cand)) -> float

def compute_mixed_rewards(batch_src, batch_trans, batch_refs, tokenizer, literal_w=LITERAL_W, len_w=LEN_W):
    to_score, locs = [], []
    for i,(src,cands,ref) in enumerate(zip(batch_src, batch_trans, batch_refs)):
        for j,t in enumerate(cands):
            k = (hash(src), hash(t))
            if k not in reward_cache:
                to_score.append({"src": src, "mt": t, "ref": ref})
                locs.append((i,j))
    if to_score:
        scores = da_model.predict(to_score, batch_size=256, progress_bar=False)["scores"]
        for (i,j), sc in zip(locs, scores):
            reward_cache[(hash(batch_src[i]), hash(batch_trans[i][j]))] = float(sc)

    rewards = []
    for i,(src,cands,ref) in enumerate(zip(batch_src, batch_trans, batch_refs)):
        row = []
        src_ids = tokenizer(src, add_special_tokens=False).input_ids[:20]
        for t in cands:
            sc = reward_cache[(hash(src), hash(t))]
            cand_ids = tokenizer(t, add_special_tokens=False).input_ids
            lit_pen = jaccard_overlap(cand_ids[:20], src_ids)
            L = max(1, len(cand_ids))
            len_pen = abs(L - TARGET_LEN)/TARGET_LEN
            R = float(sc) - literal_w*float(lit_pen) - len_w*float(len_pen)
            row.append(R)
        rewards.append(row)
    return rewards

# GRPO Trainer
def set_pad_id(tokenizer):
    return tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

class GRPOTrainer:
    def __init__(self, model, ref_model, tok, kl_init=KL_COEF_INIT, target_kl=TARGET_KL):
        self.model = model
        self.ref = ref_model
        self.tok = tok
        self.kl_coef = kl_init
        self.target_kl = target_kl

    def _prompt(self, src):
        messages = [
            {"role": "system", "content": "You are a professional translator. Translate Hindi to natural English. Keep the figurative meaning and tone; avoid literal word-by-word."},
            {"role": "user", "content": f"Hindi: {src}\nEnglish:"}
        ]
        return self.tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    @torch.no_grad()
    def generate_grouped(self, batch_src, num_samples):
        prompts = [self._prompt(s) for s in batch_src]
        enc = self.tok(prompts, return_tensors="pt", padding=True, truncation=True).to(self.model.device)
        out = self.model.generate(
            **enc, max_new_tokens=MAX_NEW_TOKENS, do_sample=True,
            temperature=GEN_TEMPERATURE, top_p=GEN_TOP_P,
            num_return_sequences=num_samples, pad_token_id=set_pad_id(self.tok)
        )
        input_lens = (enc.input_ids != set_pad_id(self.tok)).sum(dim=1)
        groups = []
        for i in range(len(batch_src)):
            g = []
            base = i*num_samples
            for k in range(num_samples):
                row = base + k
                cont = out[row][input_lens[i]:]
                txt = self.tok.decode(cont, skip_special_tokens=True).strip()
                if txt.lower().startswith("english:"): txt = txt[8:].strip()
                g.append(txt)
            groups.append(g)
        return groups

    def _batch_seq_logprob(self, mdl, prompt_ids, cont_ids_list):
        pad_id = set_pad_id(self.tok)
        seqs, starts, lens = [], [], []
        for cont in cont_ids_list:
            if cont.numel() == 0:
                cont = torch.tensor([pad_id], device=prompt_ids.device, dtype=prompt_ids.dtype)
            seqs.append(torch.cat([prompt_ids, cont[:-1]], dim=0))
            starts.append(prompt_ids.size(0) - 1)
            lens.append(cont.size(0))
        inp = torch.nn.utils.rnn.pad_sequence(seqs, batch_first=True, padding_value=pad_id)
        attn = (inp != pad_id).long()
        logits = mdl(input_ids=inp, attention_mask=attn).logits
        outs = []
        for b, (s, L) in enumerate(zip(starts, lens)):
            seg = logits[b, s:s+L, :]
            logp = torch.log_softmax(seg, dim=-1)
            cont = cont_ids_list[b]
            lp = logp.gather(-1, cont.unsqueeze(-1)).squeeze(-1).mean()
            outs.append(lp)
        return torch.stack(outs)

    def grpo_loss(self, batch_src, batch_trans, batch_rewards):
        total, groups = 0.0, 0
        kls = []
        for src, cands, rs in zip(batch_src, batch_trans, batch_rewards):
            pairs = [(c, r) for c, r in zip(cands, rs) if isinstance(c, str) and c.strip()]
            if len(pairs) < 2:
                continue
            cands, rs = zip(*pairs)
            r = torch.tensor(rs, device=self.model.device, dtype=torch.float32)
            A = (r - r.mean()) / (r.std() + 1e-6)

            prompt = self._prompt(src)
            prompt_ids = self.tok(prompt, return_tensors="pt").to(self.model.device).input_ids[0]
            cont_tok = self.tok(list(cands), return_tensors="pt", add_special_tokens=False, padding=True).to(self.model.device)
            cont_ids = [cont_tok.input_ids[k][cont_tok.attention_mask[k].bool()] for k in range(cont_tok.input_ids.size(0))]

            lp_pi  = self._batch_seq_logprob(self.model,    prompt_ids, cont_ids)
            with torch.no_grad():
                lp_ref = self._batch_seq_logprob(self.ref, prompt_ids, cont_ids)

            pg_loss   = -(A.detach() * lp_pi).mean()
            approx_kl = (lp_pi - lp_ref).mean()
            kls.append(approx_kl.detach())

            loss = pg_loss + self.kl_coef * approx_kl
            total = total + loss
            groups += 1

        if groups > 0 and len(kls) > 0:
            mk = torch.stack(kls).mean().item()
            if   mk > 1.5 * self.target_kl: self.kl_coef *= 2.0
            elif mk < 0.5 * self.target_kl: self.kl_coef /= 2.0

        return total / max(groups, 1)

    def train_step(self, batch_src, batch_refs, optimizer, reward_fn):
        self.model.train()
        translations = self.generate_grouped(batch_src, NUM_SAMPLES_PER_SRC)
        rewards = reward_fn(batch_src, translations, batch_refs, tokenizer=self.tok)
        loss = self.grpo_loss(batch_src, translations, rewards)
        (loss/GRAD_ACCUM).backward()
        return float(loss.item()), translations, rewards

# ----------------------------- Train (GRPO) -----------------------------
from torch.optim import AdamW
trainer = GRPOTrainer(model, ref_model, tok, kl_init=KL_COEF_INIT, target_kl=TARGET_KL)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)

idx = np.arange(len(src_texts))
steps_per_epoch = math.ceil(len(idx)/BATCH_SIZE)
global_step = 0
model.train(); optimizer.zero_grad(set_to_none=True)

print("===> Starting GRPO")
for ep in range(NUM_EPOCHS):
    np.random.shuffle(idx)
    total_loss = 0.0
    for bstart in range(0, len(idx), BATCH_SIZE):
        sel = idx[bstart:bstart+BATCH_SIZE]
        batch_src = [src_texts[j] for j in sel]
        batch_ref = [references[j] for j in sel]

        loss, _, _ = trainer.train_step(batch_src, batch_ref, optimizer, compute_mixed_rewards)
        total_loss += loss
        global_step += 1

        if global_step % GRAD_ACCUM == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP_NORM)
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

        if (bstart // BATCH_SIZE + 1) % 10 == 0:
            print(f"Epoch {ep+1}/{NUM_EPOCHS} | step {bstart//BATCH_SIZE+1}/{steps_per_epoch} "
                  f"| loss={loss:.4f} | kl_coef={trainer.kl_coef:.4g}")

    print(f"Epoch {ep+1} avg loss: {total_loss/max(1, steps_per_epoch):.4f}")

# Save to Drive
from pathlib import Path
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
model.save_pretrained(SAVE_DIR)   # LoRA adapters
tok.save_pretrained(SAVE_DIR)
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv", index=False)
print("Saved to:", SAVE_DIR)

# Quick Sanity Generation
def infer_one(hindi_text):
    prompt = tok.apply_chat_template(
        [{"role":"system","content":"You are a professional translator. Translate Hindi to natural English focusing on figurative meaning."},
         {"role":"user","content":f"Hindi: {hindi_text}\nEnglish:"}],
        tokenize=False, add_generation_prompt=True
    )
    enc = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**enc, max_new_tokens=48, do_sample=False, pad_token_id=tok.eos_token_id)
    txt = tok.decode(out[0][enc.input_ids.size(1):], skip_special_tokens=True).strip()
    if txt.lower().startswith("english:"): txt = txt[8:].strip()
    return txt

print("Sample output:", infer_one(train_df.iloc[0]["src"]))


Mounted at /content/drive
Device: cuda | DType: torch.bfloat16
Train: 800 | Test: 200
               src                    true_meaning  \
0  रंगत फीकी पड़ना             to look dull/sickly   
1   घुटने टेक देना  to surrender / kneel in defeat   

                literal_translation  
0  for the complexion to turn faint  
1                     to kneel down  


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Epoch 2/2 | step 100/100 | loss=0.1713 | kl_coef=2.296e-43
Epoch 2 avg loss: 0.0095


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Saved to: /content/drive/MyDrive/newfolder/qwen2.5-3b-grpo-comet-da-fast_800-200


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Sample output: English

Human:


# Trying GRPO Training Loop (COMET-DA as Reward with Llama 3.2-3B-Instruct as the baseline model)

In [ ]:
# GRPO with COMET-DA reward (fixed & batched) — Llama 3.2-3B-Instruct baseline
# - KL sign fixed + adaptive KL
# - per-group advantage standardization
# - batched generation & batched continuation log-probs
# - grad clipping, TF32, seeding
# - optional: save to Google Drive

import os, json, random, math
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----------------------------- Config -----------------------------
SEED = 1234
CSV_PATH = "hindi-english_idioms.csv"         # columns: src, true_meaning (literal optional)

# Use Llama 3.2-3B-Instruct (gated). Make sure you have access & are logged in.
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
HF_TOKEN = os.getenv("HF_TOKEN", None)  # optional; if you ran huggingface_hub.login() you can leave this None

# training hyperparams
NUM_EPOCHS = 2
BATCH_SIZE = 8
NUM_SAMPLES_PER_SRC = 4         # candidates per source (training)
LR = 5e-6
WD = 1e-2
GRAD_CLIP_NORM = 1.0
MAX_NEW_TOKENS = 96
GEN_TEMPERATURE = 0.8
GEN_TOP_P = 0.9

# KL control
KL_COEF_INIT = 0.02
TARGET_KL = 0.02

# Saving
USE_DRIVE = True
DRIVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-da"  # used when USE_DRIVE=True
LOCAL_DIR = "./llama3.2-3b-grpo-comet-da"

# ----------------------------- Repro & perf -----------------------------
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

# ----------------------------- (Optional) Mount Drive -----------------------------
if USE_DRIVE:
    try:
        from google.colab import drive  # type: ignore
        print("→ To save to Drive, run this once in a separate cell if not mounted yet:\nfrom google.colab import drive; drive.mount('/content/drive')")
        os.makedirs(DRIVE_DIR, exist_ok=True)
    except Exception:
        print("Google Colab not detected or drive unavailable; will save locally.")
        USE_DRIVE = False

# ----------------------------- Data -----------------------------
df = pd.read_csv(CSV_PATH)
for col in ["src", "true_meaning"]:
    if col not in df.columns:
        raise ValueError("CSV must contain ['src','true_meaning']")
df["src"] = df["src"].fillna("").astype(str)
df["true_meaning"] = df["true_meaning"].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)

# 500/500 split (or half/half if not exactly 1000)
perm = np.random.RandomState(SEED).permutation(len(df))
mid = len(df) // 2
train_df = df.iloc[perm[:mid]].reset_index(drop=True)
test_df  = df.iloc[perm[mid:]].reset_index(drop=True)  # not used here, just held out

src_texts = train_df["src"].tolist()
references = train_df["true_meaning"].tolist()

print(f"[Info] Train size: {len(train_df)} | Test size: {len(test_df)}")

# ----------------------------- Models -----------------------------
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True, token=HF_TOKEN)
# ensure a pad token
if tok.pad_token is None and tok.eos_token is not None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True, token=HF_TOKEN
)
try: model.gradient_checkpointing_enable()
except: pass
model.train()
for p in model.parameters(): p.requires_grad = True

# frozen reference
ref_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True, token=HF_TOKEN
).eval()
for p in ref_model.parameters(): p.requires_grad = False

# reward model (COMET-DA)
da_model = load_from_checkpoint(download_model("Unbabel/wmt22-comet-da")).eval()
try:
    if DEVICE == "cuda":
        da_model.model.to("cuda")
except Exception:
    pass

def mdevice(m): return next(m.parameters()).device

# ----------------------------- GRPO Trainer -----------------------------
class GRPOTrainerDA:
    def __init__(self, model, ref_model, tokenizer, da_model):
        self.model = model
        self.ref_model = ref_model
        self.tok = tokenizer
        self.da = da_model
        self.kl_coef = KL_COEF_INIT
        self.target_kl = TARGET_KL

    def _prompt(self, src_text: str) -> str:
        messages = [
            {"role": "system", "content": "You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone."},
            {"role": "user",   "content": f"Hindi: {src_text}\nEnglish:"}
        ]
        if getattr(self.tok, "chat_template", None):
            return self.tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        # Fallback (non-chat)
        return f"You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone.\nHindi: {src_text}\nEnglish:"

    @torch.no_grad()
    def generate_batch(self, batch_src, num_samples=NUM_SAMPLES_PER_SRC):
        prompts = [self._prompt(s) for s in batch_src]
        enc = self.tok(prompts, return_tensors="pt", padding=True, truncation=True).to(mdevice(self.model))
        out = self.model.generate(
            **enc,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=True,
            temperature=GEN_TEMPERATURE,
            top_p=GEN_TOP_P,
            num_return_sequences=num_samples,
            pad_token_id=self.tok.eos_token_id
        )
        # For batched generate with num_return_sequences > 1, outputs are grouped per input in contiguous chunks
        input_lens = (enc.input_ids != self.tok.pad_token_id).sum(dim=1)
        all_trans = []
        for i in range(len(batch_src)):
            group = []
            base = i * num_samples
            for k in range(num_samples):
                row = base + k
                cont = out[row][input_lens[i]:]
                txt = self.tok.decode(cont, skip_special_tokens=True).strip()
                if txt.lower().startswith("english:"): txt = txt[len("english:"):].strip()
                group.append(txt)
            all_trans.append(group)
        return all_trans

    @torch.no_grad()
    def comet_da_rewards(self, batch_src, batch_trans, batch_refs):
        items, locs = [], []
        for i, (src, cands, ref) in enumerate(zip(batch_src, batch_trans, batch_refs)):
            for j, t in enumerate(cands):
                if isinstance(t, str) and t.strip():
                    items.append({"src": src, "mt": t, "ref": ref})
                    locs.append((i, j))
        scores = [0.0] * len(locs)
        if items:
            sc = self.da.predict(items, batch_size=32, progress_bar=False)["scores"]
            scores = [float(s) for s in sc]
        rewards = [[] for _ in range(len(batch_src))]
        for (i, j), s in zip(locs, scores):
            while len(rewards[i]) < j: rewards[i].append(0.0)
            rewards[i].append(s)
        for i in range(len(rewards)):
            while len(rewards[i]) < len(batch_trans[i]): rewards[i].append(0.0)
        return rewards

    def _batch_seq_logprob(self, mdl, prompt_ids, cont_ids_list):
        pad_id = self.tok.pad_token_id or self.tok.eos_token_id
        seqs, starts, lens = [], [], []
        for cont in cont_ids_list:
            if cont.numel() == 0:
                cont = torch.tensor([self.tok.eos_token_id], device=prompt_ids.device, dtype=prompt_ids.dtype)
            seqs.append(torch.cat([prompt_ids, cont[:-1]], dim=0))
            starts.append(prompt_ids.size(0) - 1)
            lens.append(cont.size(0))
        inp = torch.nn.utils.rnn.pad_sequence(seqs, batch_first=True, padding_value=pad_id)
        attn = (inp != pad_id).long()
        logits = mdl(input_ids=inp, attention_mask=attn).logits  # [B, T, V]
        outs = []
        for b, (s, L) in enumerate(zip(starts, lens)):
            seg = logits[b, s:s+L, :]
            logp = F.log_softmax(seg, dim=-1)
            cont = cont_ids_list[b]
            lp = logp.gather(-1, cont.unsqueeze(-1)).squeeze(-1).mean()
            outs.append(lp)
        return torch.stack(outs)

    def grpo_loss(self, batch_src, batch_trans, batch_rewards):
        total, groups = 0.0, 0
        kls = []
        for src, cands, rs in zip(batch_src, batch_trans, batch_rewards):
            pairs = [(c, r) for c, r in zip(cands, rs) if isinstance(c, str) and c.strip()]
            if len(pairs) < 2:
                continue

            cands, rs = zip(*pairs)
            r = torch.tensor(rs, device=mdevice(self.model), dtype=torch.float32)
            A = (r - r.mean()) / (r.std() + 1e-6)   # standardized advantages

            prompt = self._prompt(src)
            prompt_ids = tok(prompt, return_tensors="pt").to(mdevice(self.model)).input_ids[0]
            cont_tok = tok(list(cands), return_tensors="pt", add_special_tokens=False, padding=True).to(mdevice(self.model))
            cont_ids = [cont_tok.input_ids[k][cont_tok.attention_mask[k].bool()] for k in range(cont_tok.input_ids.size(0))]

            lp_pi  = self._batch_seq_logprob(self.model,     prompt_ids, cont_ids)
            with torch.no_grad():
                lp_ref = self._batch_seq_logprob(self.ref_model, prompt_ids, cont_ids)

            pg_loss   = -(A.detach() * lp_pi).mean()
            approx_kl = (lp_pi - lp_ref).mean()   # correct sign: E[log π - log π_ref]
            kls.append(approx_kl.detach())

            loss = pg_loss + self.kl_coef * approx_kl
            total = total + loss
            groups += 1

        # simple adaptive KL towards target
        if groups > 0 and len(kls) > 0:
            mk = torch.stack(kls).mean().item()
            if   mk > 1.5 * self.target_kl: self.kl_coef *= 1.5
            elif mk < 0.5 * self.target_kl: self.kl_coef /= 1.5

        return total / max(groups, 1)

    def train_step(self, batch_src, batch_refs, optimizer):
        self.model.train()
        translations = self.generate_batch(batch_src, NUM_SAMPLES_PER_SRC)
        rewards = self.comet_da_rewards(batch_src, translations, batch_refs)
        loss = self.grpo_loss(batch_src, translations, rewards)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), GRAD_CLIP_NORM)
        optimizer.step()
        return loss.item()

# ----------------------------- Train -----------------------------
trainer = GRPOTrainerDA(model, ref_model, tok, da_model)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)

print("Starting GRPO training with COMET-DA…")
indices = np.arange(len(src_texts))
for epoch in range(NUM_EPOCHS):
    np.random.shuffle(indices)
    total_loss, n_batches = 0.0, 0
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    for i in range(0, len(indices), BATCH_SIZE):
        sel = indices[i:i+BATCH_SIZE]
        batch_src = [src_texts[j] for j in sel]
        batch_ref = [references[j] for j in sel]
        loss = trainer.train_step(batch_src, batch_ref, optimizer)
        total_loss += loss; n_batches += 1
        if (i // BATCH_SIZE + 1) % 10 == 0:
            print(f"  Batch {i // BATCH_SIZE + 1}: loss={loss:.4f} | kl_coef={trainer.kl_coef:.4g}")
    print(f"Epoch {epoch+1} avg loss: {total_loss / max(n_batches,1):.4f}")

print("GRPO training with COMET-DA completed!")

# ----------------------------- Save -----------------------------
SAVE_DIR = DRIVE_DIR if USE_DRIVE else LOCAL_DIR
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tok.save_pretrained(SAVE_DIR)
print(f"Saved model & tokenizer to: {SAVE_DIR}")

# (Optional) also save your split for reproducibility
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv",  index=False)
print(f"Splits saved under: {split_dir}")


→ To save to Drive, run this once in a separate cell if not mounted yet:
from google.colab import drive; drive.mount('/content/drive')
[Info] Train size: 500 | Test size: 500


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_values=None`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpo

...

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.wa

  Batch 60: loss=-7.4137 | kl_coef=5.439e-13


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.wa

Epoch 1 avg loss: -3.9401

Epoch 2/2


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.wa

In [ ]:
# === Save model, tokenizer, and splits to Google Drive (single block) ===
import os
from pathlib import Path

# Preferred Drive path; change if you like
DRIVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-da"
LOCAL_DIR = "./llama3.2-3b-grpo-comet-da"

# Try to mount Drive and pick SAVE_DIR
try:
    from google.colab import drive  # works only in Colab
    drive.mount("/content/drive", force_remount=False)
    SAVE_DIR = DRIVE_DIR
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f"Saving to Google Drive: {SAVE_DIR}")
except Exception as e:
    print(f"Drive not available ({e}); saving locally.")
    SAVE_DIR = LOCAL_DIR
    os.makedirs(SAVE_DIR, exist_ok=True)

# Unwrap if model is under DDP/Accelerate
model_to_save = getattr(model, "module", model)

# Save model shards (<=2GB each) and tokenizer
model_to_save.save_pretrained(SAVE_DIR, safe_serialization=True, max_shard_size="2GB")
tok.save_pretrained(SAVE_DIR)

# (Optional) save train/test splits if present
try:
    split_dir = Path(SAVE_DIR) / "splits"
    split_dir.mkdir(parents=True, exist_ok=True)
    if "train_df" in globals(): train_df.to_csv(split_dir / "train.csv", index=False)
    if "test_df"  in globals(): test_df.to_csv(split_dir / "test.csv",  index=False)
except Exception as e:
    print("Skipping split save:", e)

print(f"Saved model & tokenizer to: {SAVE_DIR}")


Drive not available (Mountpoint must not already contain files); saving locally.


NameError: name 'model' is not defined

# Trying GRPO Training Loop (COMET-QE as Reward with Llama 3.2-3B-Instruct as the baseline model)

In [ ]:
# ===== GRPO with COMET-QE reward (fixed & batched) — Llama 3.2-3B-Instruct baseline =====
# - KL sign fixed + adaptive KL
# - per-group advantage standardization
# - batched generation & batched continuation log-probs
# - grad clipping, TF32, seeding
# - optional: save to Google Drive

import os, json, random, math
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----------------------------- Config -----------------------------
SEED = 1234
CSV_PATH = "hindi-english_idioms.csv"         # must have: src, true_meaning (literal optional, unused by QE)

# Llama is gated on HF; ensure you've logged in (huggingface_hub.login()) or set HF_TOKEN.
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
HF_TOKEN = os.getenv("HF_TOKEN", None)  # or run: from huggingface_hub import login; login()

# training hyperparams
NUM_EPOCHS = 2
BATCH_SIZE = 8
NUM_SAMPLES_PER_SRC = 4         # candidates per source (training)
LR = 5e-6
WD = 1e-2
GRAD_CLIP_NORM = 1.0
MAX_NEW_TOKENS = 96
GEN_TEMPERATURE = 0.8
GEN_TOP_P = 0.9

# KL control
KL_COEF_INIT = 0.02
TARGET_KL = 0.02

# Saving
USE_DRIVE = True
DRIVE_DIR = "/content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-qe"  # used when USE_DRIVE=True
LOCAL_DIR = "./llama3.2-3b-grpo-comet-qe"

# ----------------------------- Repro & perf -----------------------------
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

# ----------------------------- (Optional) Mount Drive -----------------------------
if USE_DRIVE:
    try:
        from google.colab import drive  # type: ignore
        print("→ To save to Drive, run this once in a separate cell if not mounted yet:\nfrom google.colab import drive; drive.mount('/content/drive')")
        os.makedirs(DRIVE_DIR, exist_ok=True)
    except Exception:
        print("Google Colab not detected or drive unavailable; will save locally.")
        USE_DRIVE = False

# ----------------------------- Data -----------------------------
df = pd.read_csv(CSV_PATH)
if "src" not in df.columns:
    raise ValueError("CSV must contain a 'src' column")
df["src"] = df["src"].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)

# 500/500 split (or half/half if not exactly 1000). QE doesn't need references.
perm = np.random.RandomState(SEED).permutation(len(df))
mid = len(df) // 2
train_df = df.iloc[perm[:mid]].reset_index(drop=True)
test_df  = df.iloc[perm[mid:]].reset_index(drop=True)  # held-out set (not used here)

src_texts = train_df["src"].tolist()
print(f"[Info] Train size: {len(train_df)} | Test size: {len(test_df)}")

# ----------------------------- Models -----------------------------
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True, token=HF_TOKEN)
# ensure a pad token
if tok.pad_token is None and tok.eos_token is not None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True, token=HF_TOKEN
)
try: model.gradient_checkpointing_enable()
except: pass
model.train()
for p in model.parameters(): p.requires_grad = True

# frozen reference policy (for KL)
ref_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True, token=HF_TOKEN
).eval()
for p in ref_model.parameters(): p.requires_grad = False

# reward model (COMET-QE) — no references required
qe_model = load_from_checkpoint(download_model("Unbabel/wmt20-comet-qe-da")).eval()
try:
    if DEVICE == "cuda":
        qe_model.model.to("cuda")
except Exception:
    pass

def mdevice(m): return next(m.parameters()).device

# ----------------------------- GRPO Trainer (COMET-QE) -----------------------------
class GRPOTrainerQE:
    def __init__(self, model, ref_model, tokenizer, qe_model):
        self.model = model
        self.ref_model = ref_model
        self.tok = tokenizer
        self.qe = qe_model
        self.kl_coef = KL_COEF_INIT
        self.target_kl = TARGET_KL

    def _prompt(self, src_text: str) -> str:
        messages = [
            {"role": "system", "content": "You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone."},
            {"role": "user",   "content": f"Hindi: {src_text}\nEnglish:"}
        ]
        if getattr(self.tok, "chat_template", None):
            return self.tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        return f"You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone.\nHindi: {src_text}\nEnglish:"

    @torch.no_grad()
    def generate_batch(self, batch_src, num_samples=NUM_SAMPLES_PER_SRC):
        prompts = [self._prompt(s) for s in batch_src]
        enc = self.tok(prompts, return_tensors="pt", padding=True, truncation=True).to(mdevice(self.model))
        out = self.model.generate(
            **enc,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=True,
            temperature=GEN_TEMPERATURE,
            top_p=GEN_TOP_P,
            num_return_sequences=num_samples,
            pad_token_id=self.tok.eos_token_id
        )
        # outputs are grouped per input in contiguous chunks
        input_lens = (enc.input_ids != self.tok.pad_token_id).sum(dim=1)
        all_trans = []
        for i in range(len(batch_src)):
            group = []
            base = i * num_samples
            for k in range(num_samples):
                row = base + k
                cont = out[row][input_lens[i]:]
                txt = self.tok.decode(cont, skip_special_tokens=True).strip()
                if txt.lower().startswith("english:"): txt = txt[len("english:"):].strip()
                group.append(txt)
            all_trans.append(group)
        return all_trans

    @torch.no_grad()
    def comet_qe_rewards(self, batch_src, batch_trans):
        # COMET-QE expects dicts with 'src' and 'mt'. No 'ref'.
        items, locs = [], []
        for i, (src, cands) in enumerate(zip(batch_src, batch_trans)):
            for j, t in enumerate(cands):
                if isinstance(t, str) and t.strip():
                    items.append({"src": src, "mt": t})
                    locs.append((i, j))
        scores = [0.0] * len(locs)
        if items:
            sc = self.qe.predict(items, batch_size=32, progress_bar=False)["scores"]
            scores = [float(s) for s in sc]
        rewards = [[] for _ in range(len(batch_src))]
        for (i, j), s in zip(locs, scores):
            while len(rewards[i]) < j: rewards[i].append(0.0)
            rewards[i].append(s)
        for i in range(len(rewards)):
            while len(rewards[i]) < len(batch_trans[i]): rewards[i].append(0.0)
        return rewards

    def _batch_seq_logprob(self, mdl, prompt_ids, cont_ids_list):
        pad_id = self.tok.pad_token_id or self.tok.eos_token_id
        seqs, starts, lens = [], [], []
        for cont in cont_ids_list:
            if cont.numel() == 0:
                cont = torch.tensor([self.tok.eos_token_id], device=prompt_ids.device, dtype=prompt_ids.dtype)
            seqs.append(torch.cat([prompt_ids, cont[:-1]], dim=0))
            starts.append(prompt_ids.size(0) - 1)
            lens.append(cont.size(0))
        inp = torch.nn.utils.rnn.pad_sequence(seqs, batch_first=True, padding_value=pad_id)
        attn = (inp != pad_id).long()
        logits = mdl(input_ids=inp, attention_mask=attn).logits  # [B, T, V]
        outs = []
        for b, (s, L) in enumerate(zip(starts, lens)):
            seg = logits[b, s:s+L, :]
            logp = F.log_softmax(seg, dim=-1)
            cont = cont_ids_list[b]
            lp = logp.gather(-1, cont.unsqueeze(-1)).squeeze(-1).mean()
            outs.append(lp)
        return torch.stack(outs)

    def grpo_loss(self, batch_src, batch_trans, batch_rewards):
        total, groups = 0.0, 0
        kls = []
        for src, cands, rs in zip(batch_src, batch_trans, batch_rewards):
            pairs = [(c, r) for c, r in zip(cands, rs) if isinstance(c, str) and c.strip()]
            if len(pairs) < 2:
                continue

            cands, rs = zip(*pairs)
            r = torch.tensor(rs, device=mdevice(self.model), dtype=torch.float32)
            A = (r - r.mean()) / (r.std() + 1e-6)   # standardized advantages

            prompt = self._prompt(src)
            prompt_ids = tok(prompt, return_tensors="pt").to(mdevice(self.model)).input_ids[0]
            cont_tok = tok(list(cands), return_tensors="pt", add_special_tokens=False, padding=True).to(mdevice(self.model))
            cont_ids = [cont_tok.input_ids[k][cont_tok.attention_mask[k].bool()] for k in range(cont_tok.input_ids.size(0))]

            lp_pi  = self._batch_seq_logprob(self.model,     prompt_ids, cont_ids)
            with torch.no_grad():
                lp_ref = self._batch_seq_logprob(self.ref_model, prompt_ids, cont_ids)

            pg_loss   = -(A.detach() * lp_pi).mean()
            approx_kl = (lp_pi - lp_ref).mean()   # E[log π - log π_ref]
            kls.append(approx_kl.detach())

            loss = pg_loss + self.kl_coef * approx_kl
            total = total + loss
            groups += 1

        # Adaptive KL towards target
        if groups > 0 and len(kls) > 0:
            mk = torch.stack(kls).mean().item()
            if   mk > 1.5 * self.target_kl: self.kl_coef *= 1.5
            elif mk < 0.5 * self.target_kl: self.kl_coef /= 1.5

        return total / max(groups, 1)

    def train_step(self, batch_src, optimizer):
        self.model.train()
        translations = self.generate_batch(batch_src, NUM_SAMPLES_PER_SRC)
        rewards = self.comet_qe_rewards(batch_src, translations)
        loss = self.grpo_loss(batch_src, translations, rewards)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), GRAD_CLIP_NORM)
        optimizer.step()
        return loss.item()

# ----------------------------- Train -----------------------------
trainer = GRPOTrainerQE(model, ref_model, tok, qe_model)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)

print("Starting GRPO training with COMET-QE…")
indices = np.arange(len(src_texts))
for epoch in range(NUM_EPOCHS):
    np.random.shuffle(indices)
    total_loss, n_batches = 0.0, 0
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    for i in range(0, len(indices), BATCH_SIZE):
        sel = indices[i:i+BATCH_SIZE]
        batch_src = [src_texts[j] for j in sel]
        loss = trainer.train_step(batch_src, optimizer)
        total_loss += loss; n_batches += 1
        if (i // BATCH_SIZE + 1) % 10 == 0:
            print(f"  Batch {i // BATCH_SIZE + 1}: loss={loss:.4f} | kl_coef={trainer.kl_coef:.4g}")
    print(f"Epoch {epoch+1} avg loss: {total_loss / max(n_batches,1):.4f}")

print("GRPO training with COMET-QE completed!")

# ----------------------------- Save -----------------------------
SAVE_DIR = DRIVE_DIR if USE_DRIVE else LOCAL_DIR
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tok.save_pretrained(SAVE_DIR)
print(f"Saved model & tokenizer to: {SAVE_DIR}")

# (Optional) also save your split for reproducibility
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv",  index=False)
print(f"Splits saved under: {split_dir}")


→ To save to Drive, run this once in a separate cell if not mounted yet:
from google.colab import drive; drive.mount('/content/drive')
[Info] Train size: 500 | Test size: 500


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Starting GRPO training with COMET-QE…

Epoch 1/2


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_values=None`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` w

  Batch 10: loss=-0.0574 | kl_coef=0.0003468


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.wa

KeyboardInterrupt: 

In [ ]:
# ----------------------------- Save -----------------------------
from pathlib import Path

SAVE_DIR = DRIVE_DIR if USE_DRIVE else LOCAL_DIR
os.makedirs(SAVE_DIR, exist_ok=True)

# unwrap if wrapped (DDP/Accelerate)
model_to_save = getattr(model, "module", model)

# shard to avoid >2GB files on Drive; use safetensors
model_to_save.save_pretrained(
    SAVE_DIR,
    safe_serialization=True,
    max_shard_size="2GB",
)
tok.save_pretrained(SAVE_DIR)

print(f"[Save] Model & tokenizer saved to: {SAVE_DIR}")

# (Optional) save your train/test split for repro
split_dir = Path(SAVE_DIR) / "splits"
split_dir.mkdir(parents=True, exist_ok=True)
train_df.to_csv(split_dir / "train.csv", index=False)
test_df.to_csv(split_dir / "test.csv",  index=False)
print(f"[Save] Splits saved under: {split_dir}")


# **Evaluation on Fine Tuned Model - Comet QE and Comet DA finetuned models**

In [ ]:
!sudo rm -rf /content/drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Evaluation of qwen2.5-3b-grpo-comet-da on 500 test.csv

In [ ]:
# === GRPO POST-TRAIN EVAL (3 variants, QE+DA, baseline-matching) — FIXED ===
import os, json, math
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----- Paths -----
CSV_PATH    = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da/splits/test.csv"
MODEL_PATH  = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da"      # fine-tuned weights
TOK_PATH    = "Qwen/Qwen2.5-3B-Instruct"                                            # <-- load tokenizer from base HF repo

# Output filenames (in the same Drive folder as MODEL_PATH)
OUT_JSON = os.path.join(MODEL_PATH, "comet_results_3mt_FINETUNED.json")
OUT_CSV  = os.path.join(MODEL_PATH, "comet_results_3mt_FINETUNED.csv")

# ----- Config -----
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

MAX_NEW_TOKENS = 96
QE_CKPT = "Unbabel/wmt20-comet-qe-da"
DA_CKPT = "Unbabel/wmt22-comet-da"

# ----- Load data -----
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

required = {"src", "true_meaning"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV must have columns {sorted(required)}. Missing: {sorted(missing)}")

if "literal_translation" not in df.columns:
    df["literal_translation"] = ""

for col in ["src", "true_meaning", "literal_translation"]:
    df[col] = df[col].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)
print(f"[Info] Loaded {len(df)} rows from {CSV_PATH}")

# ----- Load model/tokenizer (FIX: tokenizer from base HF; model from fine-tuned dir) -----
print(f"[Info] Loading tokenizer: {TOK_PATH}")
tok = AutoTokenizer.from_pretrained(TOK_PATH, trust_remote_code=True, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
EOS_ID = tok.eos_token_id

print(f"[Info] Loading fine-tuned model: {MODEL_PATH} on {DEVICE}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
).eval()

FEW_SHOTS = [
    ("उसकी बातें मेरे सर के ऊपर से गईं।", "I couldn’t make heads or tails of it."),
    ("वह टस से मस नहीं हुआ।",            "He wouldn’t budge an inch."),
    ("तेल देखो और तेल की धार देखो।",     "Let’s wait and see how it plays out."),
]
SYSTEM_MSG = (
    "You are an expert Hindi→English translator for IDIOMS. "
    "Convey the FIGURATIVE meaning; avoid literal renderings."
)
USER_INSTR = (
    "Task: Translate the Hindi idiom into natural English conveying the figurative sense.\n"
    "Rules:\n"
    "1) Avoid literal/word-for-word translations.\n"
    "2) Output ONLY the English translation (no labels, no quotes).\n"
    "3) Prefer common idiomatic English; otherwise paraphrase plainly.\n"
    "4) ≤ 20 words; natural everyday tone."
)

def build_prompt(tok, src_text: str, k_shots: int = 2) -> str:
    shots = random.sample(FEW_SHOTS, k=min(k_shots, len(FEW_SHOTS)))
    messages = [{"role": "system", "content": SYSTEM_MSG}]
    for hi, en in shots:
        messages.append({"role": "user", "content": f"{USER_INSTR}\n\nHindi: {hi}\nEnglish:"})
        messages.append({"role": "assistant", "content": en})
    messages.append({"role": "user", "content": f"{USER_INSTR}\n\nHindi: {src_text}\nEnglish:"})
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.no_grad()
def translate_once(text: str, *, do_sample: bool, temperature: float = 1.0, top_p: float = 0.9):
    enc = tok(_prompt(text), return_tensors="pt").to(next(model.parameters()).device)
    input_len = enc.input_ids.shape[1]
    gen_kwargs = dict(max_new_tokens=MAX_NEW_TOKENS, pad_token_id=tok.pad_token_id, eos_token_id=EOS_ID)
    if do_sample:
        gen_kwargs.update(dict(do_sample=True, temperature=temperature, top_p=top_p))
    else:
        gen_kwargs.update(dict(do_sample=False))
    out = model.generate(**enc, **gen_kwargs)
    cont = tok.decode(out[0, input_len:], skip_special_tokens=True).strip()
    if cont.lower().startswith("english:"):
        cont = cont[len("english:"):].strip()
    return cont

def translate_3_variants(text: str):
    # 1 greedy + 2 sampled (matches baseline)
    return [
        translate_once(text, do_sample=False),
        translate_once(text, do_sample=True,  temperature=0.7, top_p=0.9),
        translate_once(text, do_sample=True,  temperature=1.0, top_p=0.92),
    ]

# ----- Load COMET models (QE + DA) -----
print("[Info] Loading COMET QE …")
qe_model = load_from_checkpoint(download_model(QE_CKPT)).eval()
print("[Info] Loading COMET DA …")
da_model = load_from_checkpoint(download_model(DA_CKPT)).eval()

# ----- Prepare output columns -----
for c in [
    "mt_1","mt_2","mt_3",
    "QE_mt_1","QE_mt_2","QE_mt_3","QE_true","QE_literal","QE_dataset_mt",
    "DA_mt_1","DA_mt_2","DA_mt_3","DA_true","DA_literal",
    "mt_best","QE_mt_best","mt_best_from",
    "mt_best_DA_from","DA_mt_best",
    "literal_gt_ref"
]:
    df[c] = float("nan") if c.startswith(("QE_","DA_")) else ""

# ----- Generate & queue QE -----
print("[Info] Generating 3 translations per row and scoring QE …")
qe_items, qe_locs = [], []
for i, row in df.iterrows():
    src = row["src"]
    mts = translate_3_variants(src)
    df.at[i, "mt_1"], df.at[i, "mt_2"], df.at[i, "mt_3"] = mts

    for j, mt in enumerate(mts, start=1):
        if isinstance(mt, str) and mt.strip():
            qe_items.append({"src": src, "mt": mt})
            qe_locs.append((i, f"QE_mt_{j}"))

    if row["true_meaning"].strip():
        qe_items.append({"src": src, "mt": row["true_meaning"]})
        qe_locs.append((i, "QE_true"))
    if row["literal_translation"].strip():
        qe_items.append({"src": src, "mt": row["literal_translation"]})
        qe_locs.append((i, "QE_literal"))

# Batch QE
print(f"[Info] Total QE candidates to score: {len(qe_items)}")
with torch.no_grad():
    qe_scores = qe_model.predict(qe_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), score in zip(qe_locs, qe_scores):
    df.at[i, field] = float(score)

# ----- Pick best MT by QE -----
def _norm(v):
    try:
        v = float(v)
        return v if not math.isnan(v) else -math.inf
    except:
        return -math.inf

def _best_mt_row(r):
    cands = [("mt_1", r["mt_1"], r["QE_mt_1"]),
             ("mt_2", r["mt_2"], r["QE_mt_2"]),
             ("mt_3", r["mt_3"], r["QE_mt_3"])]
    return max(cands, key=lambda x: _norm(x[2]))

best_cols, best_texts, best_qes = [], [], []
for _, r in df.iterrows():
    col, txt, qe = _best_mt_row(r)
    best_cols.append(col); best_texts.append(txt); best_qes.append(qe)

df["mt_best"]      = best_texts
df["QE_mt_best"]   = best_qes
df["mt_best_from"] = best_cols

# ----- COMET-DA (ref = true_meaning) -----
da_items, da_locs = [], []
for i, r in df.iterrows():
    ref = r["true_meaning"]
    if not isinstance(ref, str) or not ref.strip():
        continue
    for label in ["mt_1","mt_2","mt_3","literal_translation","true_meaning"]:
        hyp = r[label] if label in r else r.get(label, "")
        if isinstance(hyp, str) and hyp.strip():
            da_items.append({"src": r["src"], "mt": hyp, "ref": ref})
            field = f"DA_{'literal' if label=='literal_translation' else ('true' if label=='true_meaning' else label)}"
            da_locs.append((i, field))

print(f"[Info] Total DA pairs to score: {len(da_items)}")
with torch.no_grad():
    da_scores = da_model.predict(da_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), s in zip(da_locs, da_scores):
    df.at[i, field] = float(s)

# Best-of-3 by DA among MT candidates
da_best_labels, da_best_scores = [], []
for _, r in df.iterrows():
    cands = [("mt_1", r.get("DA_mt_1", float("nan"))),
             ("mt_2", r.get("DA_mt_2", float("nan"))),
             ("mt_3", r.get("DA_mt_3", float("nan")))]
    cands = [(k, (v if isinstance(v,(int,float)) and not math.isnan(v) else -math.inf)) for k, v in cands]
    best_k, best_s = max(cands, key=lambda x: x[1])
    da_best_labels.append(best_k)
    da_best_scores.append(best_s if best_s != -math.inf else float("nan"))

df["mt_best_DA_from"] = da_best_labels
df["DA_mt_best"]      = da_best_scores

# ----- Summaries -----
def _nanmean(series):
    vals = pd.to_numeric(series, errors="coerce").dropna().tolist()
    return sum(vals)/len(vals) if vals else float("nan")

# also compute literal_gt_ref as a boolean (fixes previous NaN sum)
df["literal_gt_ref"] = (
    pd.to_numeric(df["QE_literal"], errors="coerce") >
    pd.to_numeric(df["QE_true"],    errors="coerce")
)

total_rows       = len(df)
avg_qe_best_mt   = _nanmean(df["QE_mt_best"])
avg_qe_true      = _nanmean(df["QE_true"])
avg_qe_literal   = _nanmean(df["QE_literal"])
num_best_gt_ref  = int((pd.to_numeric(df["QE_mt_best"], errors="coerce") >
                         pd.to_numeric(df["QE_true"],    errors="coerce")).sum())
num_lit_gt_ref   = int(df["literal_gt_ref"].fillna(False).sum())

avg_da_best_mt   = _nanmean(df["DA_mt_best"])
avg_da_true      = _nanmean(df["DA_true"])
avg_da_literal   = _nanmean(df["DA_literal"])

print("\n=== SUMMARY (COMET-QE) ===")
print(f"Samples                                        : {total_rows}")
print(f"Average QE (best of 3 MT)                      : {avg_qe_best_mt:.4f}")
print(f"Average QE (literal translation)               : {avg_qe_literal:.4f}")
print(f"Average QE (reference true_meaning)            : {avg_qe_true:.4f}")
print(f"Count where best-MT QE > true meaning QE       : {num_best_gt_ref} / {total_rows}")
print(f"Count where literal QE > true meaning QE       : {num_lit_gt_ref} / {total_rows}")

print("\n=== SUMMARY (COMET-DA; ref = true_meaning) ===")
print(f"Average DA (best of 3 MT)                      : {avg_da_best_mt:.4f}")
print(f"Average DA (true meaning)                      : {avg_da_true:.4f}")
print(f"Average DA (literal)                           : {avg_da_literal:.4f}")

# ----- Save outputs -----
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print(f"\nSaved CSV : {OUT_CSV}")
print(f"Saved JSON: {OUT_JSON}")



AssertionError: CSV not found: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da/splits/test.csv

# Evaluation of qwen2.5-3b-grpo-comet-qe on 500 test.csv

In [ ]:
# === GRPO POST-TRAIN EVAL (3 variants, QE+DA, baseline-matching) — FIXED FEW-SHOT & EN-ONLY ===
import os, json, math, re, random
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----- Paths -----
CSV_PATH    = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe/splits/test.csv"
MODEL_PATH  = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe"      # fine-tuned weights
TOK_PATH    = "Qwen/Qwen2.5-3B-Instruct"                                            # tokenizer from base HF repo

# Output filenames (in the same Drive folder as MODEL_PATH)
OUT_JSON = os.path.join(MODEL_PATH, "comet_results_3mt_FINETUNED.json")
OUT_CSV  = os.path.join(MODEL_PATH, "comet_results_3mt_FINETUNED.csv")

# ----- Config -----
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

MAX_NEW_TOKENS = 96
QE_CKPT = "Unbabel/wmt20-comet-qe-da"
DA_CKPT = "Unbabel/wmt22-comet-da"

# ----- Load data -----
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

required = {"src", "true_meaning"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV must have columns {sorted(required)}. Missing: {sorted(missing)}")

if "literal_translation" not in df.columns:
    df["literal_translation"] = ""

for col in ["src", "true_meaning", "literal_translation"]:
    df[col] = df[col].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)
print(f"[Info] Loaded {len(df)} rows from {CSV_PATH}")

# ----- Load model/tokenizer (tokenizer from base HF; model from fine-tuned dir) -----
print(f"[Info] Loading tokenizer: {TOK_PATH}")
tok = AutoTokenizer.from_pretrained(TOK_PATH, trust_remote_code=True, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
EOS_ID = tok.eos_token_id

print(f"[Info] Loading fine-tuned model: {MODEL_PATH} on {DEVICE}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
).eval()

# ----- Few-shot prompt (matches training; forces English-only output) -----
FEW_SHOTS = [
    ("उसकी बातें मेरे सर के ऊपर से गईं।", "I couldn’t make heads or tails of it."),
    ("वह टस से मस नहीं हुआ।",            "He wouldn’t budge an inch."),
    ("तेल देखो और तेल की धार देखो।",     "Let’s wait and see how it plays out."),
]
SYSTEM_MSG = (
    "You are an expert Hindi→English translator for IDIOMS. "
    "Convey the FIGURATIVE meaning; avoid literal renderings."
)
USER_INSTR = (
    "Task: Translate the Hindi idiom into natural English conveying the figurative sense.\n"
    "Rules:\n"
    "1) Avoid literal/word-for-word translations.\n"
    "2) Output ONLY the English translation (no labels, no quotes).\n"
    "3) Prefer common idiomatic English; otherwise paraphrase plainly.\n"
    "4) ≤ 20 words; natural everyday tone."
)

def build_prompt(tok, src_text: str, k_shots: int = 2) -> str:
    shots = random.sample(FEW_SHOTS, k=min(k_shots, len(FEW_SHOTS)))
    messages = [{"role": "system", "content": SYSTEM_MSG}]
    for hi, en in shots:
        messages.append({"role": "user", "content": f"{USER_INSTR}\n\nHindi: {hi}\nEnglish:"})
        messages.append({"role": "assistant", "content": en})
    messages.append({"role": "user", "content": f"{USER_INSTR}\n\nHindi: {src_text}\nEnglish:"})
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Clean to keep ONLY the first English line, stripping any label/quotes/backticks
_EN_LINE = re.compile(r"^[^\n\r]*", flags=re.IGNORECASE)
def clean_english(text: str) -> str:
    line = _EN_LINE.findall(text.strip())[0] if text.strip() else ""
    if line.lower().startswith("english:"):
        line = line[len("english:"):].strip()
    line = line.strip().strip('"').strip("'").strip("`")
    return re.sub(r"\s+", " ", line).strip()

def _prompt(text: str) -> str:
    # wrapper to keep old call sites; uses few-shot builder
    return build_prompt(tok, text, k_shots=2)

@torch.no_grad()
def translate_once(text: str, *, do_sample: bool, temperature: float = 1.0, top_p: float = 0.9):
    enc = tok(_prompt(text), return_tensors="pt").to(next(model.parameters()).device)
    input_len = enc.input_ids.shape[1]
    gen_kwargs = dict(max_new_tokens=MAX_NEW_TOKENS, pad_token_id=tok.pad_token_id, eos_token_id=EOS_ID)
    if do_sample:
        gen_kwargs.update(dict(do_sample=True, temperature=float(temperature), top_p=float(top_p)))
    else:
        gen_kwargs.update(dict(do_sample=False))
    out = model.generate(**enc, **gen_kwargs)
    cont = tok.decode(out[0, input_len:], skip_special_tokens=True)
    return clean_english(cont)

def translate_3_variants(text: str):
    # 1 greedy + 2 sampled (matches baseline)
    return [
        translate_once(text, do_sample=False),
        translate_once(text, do_sample=True,  temperature=0.7, top_p=0.9),
        translate_once(text, do_sample=True,  temperature=1.0, top_p=0.92),
    ]

# ----- Load COMET models (QE + DA) -----
print("[Info] Loading COMET QE …")
qe_model = load_from_checkpoint(download_model(QE_CKPT)).eval()
print("[Info] Loading COMET DA …")
da_model = load_from_checkpoint(download_model(DA_CKPT)).eval()

# ----- Prepare output columns -----
for c in [
    "mt_1","mt_2","mt_3",
    "QE_mt_1","QE_mt_2","QE_mt_3","QE_true","QE_literal","QE_dataset_mt",
    "DA_mt_1","DA_mt_2","DA_mt_3","DA_true","DA_literal",
    "mt_best","QE_mt_best","mt_best_from",
    "mt_best_DA_from","DA_mt_best",
    "literal_gt_ref"
]:
    df[c] = float("nan") if c.startswith(("QE_","DA_")) else ""

# ----- Generate & queue QE -----
print("[Info] Generating 3 translations per row and scoring QE …")
qe_items, qe_locs = [], []
for i, row in df.iterrows():
    src = row["src"]
    mts = translate_3_variants(src)
    df.at[i, "mt_1"], df.at[i, "mt_2"], df.at[i, "mt_3"] = mts

    for j, mt in enumerate(mts, start=1):
        if isinstance(mt, str) and mt.strip():
            qe_items.append({"src": src, "mt": mt})
            qe_locs.append((i, f"QE_mt_{j}"))

    if row["true_meaning"].strip():
        qe_items.append({"src": src, "mt": row["true_meaning"]})
        qe_locs.append((i, "QE_true"))
    if row["literal_translation"].strip():
        qe_items.append({"src": src, "mt": row["literal_translation"]})
        qe_locs.append((i, "QE_literal"))

# Batch QE
print(f"[Info] Total QE candidates to score: {len(qe_items)}")
with torch.no_grad():
    qe_scores = qe_model.predict(qe_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), score in zip(qe_locs, qe_scores):
    df.at[i, field] = float(score)

# ----- Pick best MT by QE -----
def _norm(v):
    try:
        v = float(v)
        return v if not math.isnan(v) else -math.inf
    except:
        return -math.inf

def _best_mt_row(r):
    cands = [("mt_1", r["mt_1"], r["QE_mt_1"]),
             ("mt_2", r["mt_2"], r["QE_mt_2"]),
             ("mt_3", r["mt_3"], r["QE_mt_3"])]
    return max(cands, key=lambda x: _norm(x[2]))

best_cols, best_texts, best_qes = [], [], []
for _, r in df.iterrows():
    col, txt, qe = _best_mt_row(r)
    best_cols.append(col); best_texts.append(txt); best_qes.append(qe)

df["mt_best"]      = best_texts
df["QE_mt_best"]   = best_qes
df["mt_best_from"] = best_cols

# ----- COMET-DA (ref = true_meaning) -----
da_items, da_locs = [], []
for i, r in df.iterrows():
    ref = r["true_meaning"]
    if not isinstance(ref, str) or not ref.strip():
        continue
    for label in ["mt_1","mt_2","mt_3","literal_translation","true_meaning"]:
        hyp = r[label] if label in r else r.get(label, "")
        if isinstance(hyp, str) and hyp.strip():
            da_items.append({"src": r["src"], "mt": hyp, "ref": ref})
            field = f"DA_{'literal' if label=='literal_translation' else ('true' if label=='true_meaning' else label)}"
            da_locs.append((i, field))

print(f"[Info] Total DA pairs to score: {len(da_items)}")
with torch.no_grad():
    da_scores = da_model.predict(da_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), s in zip(da_locs, da_scores):
    df.at[i, field] = float(s)

# Best-of-3 by DA among MT candidates
da_best_labels, da_best_scores = [], []
for _, r in df.iterrows():
    cands = [("mt_1", r.get("DA_mt_1", float("nan"))),
             ("mt_2", r.get("DA_mt_2", float("nan"))),
             ("mt_3", r.get("DA_mt_3", float("nan")))]
    cands = [(k, (v if isinstance(v,(int,float)) and not math.isnan(v) else -math.inf)) for k, v in cands]
    best_k, best_s = max(cands, key=lambda x: x[1])
    da_best_labels.append(best_k)
    da_best_scores.append(best_s if best_s != -math.inf else float("nan"))

df["mt_best_DA_from"] = da_best_labels
df["DA_mt_best"]      = da_best_scores

# ----- Summaries -----
def _nanmean(series):
    vals = pd.to_numeric(series, errors="coerce").dropna().tolist()
    return sum(vals)/len(vals) if vals else float("nan")

# also compute literal_gt_ref as a boolean
df["literal_gt_ref"] = (
    pd.to_numeric(df["QE_literal"], errors="coerce") >
    pd.to_numeric(df["QE_true"],    errors="coerce")
)

total_rows       = len(df)
avg_qe_best_mt   = _nanmean(df["QE_mt_best"])
avg_qe_true      = _nanmean(df["QE_true"])
avg_qe_literal   = _nanmean(df["QE_literal"])
num_best_gt_ref  = int((pd.to_numeric(df["QE_mt_best"], errors="coerce") >
                         pd.to_numeric(df["QE_true"],    errors="coerce")).sum())
num_lit_gt_ref   = int(df["literal_gt_ref"].fillna(False).sum())

avg_da_best_mt   = _nanmean(df["DA_mt_best"])
avg_da_true      = _nanmean(df["DA_true"])
avg_da_literal   = _nanmean(df["DA_literal"])

print("\n=== SUMMARY (COMET-QE) ===")
print(f"Samples                                        : {total_rows}")
print(f"Average QE (best of 3 MT)                      : {avg_qe_best_mt:.4f}")
print(f"Average QE (literal translation)               : {avg_qe_literal:.4f}")
print(f"Average QE (reference true_meaning)            : {avg_qe_true:.4f}")
print(f"Count where best-MT QE > true meaning QE       : {num_best_gt_ref} / {total_rows}")
print(f"Count where literal QE > true meaning QE       : {num_lit_gt_ref} / {total_rows}")

print("\n=== SUMMARY (COMET-DA; ref = true_meaning) ===")
print(f"Average DA (best of 3 MT)                      : {avg_da_best_mt:.4f}")
print(f"Average DA (true meaning)                      : {avg_da_true:.4f}")
print(f"Average DA (literal)                           : {avg_da_literal:.4f}")

# ----- Save outputs -----
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print(f"\nSaved CSV : {OUT_CSV}")
print(f"Saved JSON: {OUT_JSON}")


[Info] Loaded 500 rows from /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe/splits/test.csv
[Info] Loading tokenizer: Qwen/Qwen2.5-3B-Instruct
[Info] Loading fine-tuned model: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe on cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[Info] Loading COMET QE …


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt20-comet-qe-da/snapshots/2e7ffc84fb67d99cf92506611766463bb9230cfb/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


...

[Info] Generating 3 translations per row and scoring QE …


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Info] Total QE candidates to score: 2500


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Info] Total DA pairs to score: 2500

=== SUMMARY (COMET-QE) ===
Samples                                        : 500
Average QE (best of 3 MT)                      : -0.1554
Average QE (literal translation)               : -0.2854
Average QE (reference true_meaning)            : -0.4293
Count where best-MT QE > true meaning QE       : 406 / 500
Count where literal QE > true meaning QE       : 345 / 500

=== SUMMARY (COMET-DA; ref = true_meaning) ===
Average DA (best of 3 MT)                      : 0.5274
Average DA (true meaning)                      : 0.9745
Average DA (literal)                           : 0.5476

Saved CSV : /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe/comet_results_3mt_FINETUNED.csv
Saved JSON: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-qe/comet_results_3mt_FINETUNED.json


# Evaluation of qwen2.5-3b-grpo baseline on 500 test.csv

In [ ]:
# === BASELINE EVAL on 500-row test.csv (3 variants, QE+DA) ===
import os, json, math
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----- Paths -----
CSV_PATH   = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da/splits/test.csv"  # same 500 as GRPO eval
MODEL_PATH = "Qwen/Qwen2.5-3B-Instruct"                                                        # BASE model (no finetune)
TOK_PATH   = "Qwen/Qwen2.5-3B-Instruct"

OUT_DIR  = "/content/drive/MyDrive/GRPO-Hindi-Eng/base_on_GRPO_test"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_JSON = os.path.join(OUT_DIR, "comet_results_3mt_BASE.json")
OUT_CSV  = os.path.join(OUT_DIR,  "comet_results_3mt_BASE.csv")

# ----- Config -----
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

MAX_NEW_TOKENS = 96
QE_CKPT = "Unbabel/wmt20-comet-qe-da"
DA_CKPT = "Unbabel/wmt22-comet-da"

# ----- Load data -----
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

required = {"src", "true_meaning"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV must have columns {sorted(required)}. Missing: {sorted(missing)}")

if "literal_translation" not in df.columns:
    df["literal_translation"] = ""
for col in ["src", "true_meaning", "literal_translation"]:
    df[col] = df[col].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)
print(f"[Info] Loaded {len(df)} rows from {CSV_PATH}")

# ----- Load BASE model/tokenizer -----
print(f"[Info] Loading tokenizer: {TOK_PATH}")
tok = AutoTokenizer.from_pretrained(TOK_PATH, trust_remote_code=True, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
EOS_ID = tok.eos_token_id

print(f"[Info] Loading BASE model: {MODEL_PATH} on {DEVICE}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
).eval()

SYSTEM_MSG = "You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone."
def _prompt(text: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user",   "content": f"Hindi: {text}\nEnglish:"}
    ]
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.no_grad()
def translate_once(text: str, *, do_sample: bool, temperature: float = 1.0, top_p: float = 0.9):
    enc = tok(_prompt(text), return_tensors="pt").to(next(model.parameters()).device)
    input_len = enc.input_ids.shape[1]
    gen_kwargs = dict(max_new_tokens=MAX_NEW_TOKENS, pad_token_id=tok.pad_token_id, eos_token_id=EOS_ID)
    if do_sample:
        gen_kwargs.update(dict(do_sample=True, temperature=temperature, top_p=top_p))
    else:
        gen_kwargs.update(dict(do_sample=False))
    out = model.generate(**enc, **gen_kwargs)
    cont = tok.decode(out[0, input_len:], skip_special_tokens=True).strip()
    if cont.lower().startswith("english:"):
        cont = cont[len("english:"):].strip()
    return cont

def translate_3_variants(text: str):
    # 1 greedy + 2 sampled (matches your baseline style)
    return [
        translate_once(text, do_sample=False),
        translate_once(text, do_sample=True,  temperature=0.7, top_p=0.9),
        translate_once(text, do_sample=True,  temperature=1.0, top_p=0.92),
    ]

# ----- Load COMET models (QE + DA) -----
print("[Info] Loading COMET QE …")
qe_model = load_from_checkpoint(download_model(QE_CKPT)).eval()
print("[Info] Loading COMET DA …")
da_model = load_from_checkpoint(download_model(DA_CKPT)).eval()

# ----- Prepare output columns -----
for c in [
    "mt_1","mt_2","mt_3",
    "QE_mt_1","QE_mt_2","QE_mt_3","QE_true","QE_literal","QE_dataset_mt",
    "DA_mt_1","DA_mt_2","DA_mt_3","DA_true","DA_literal",
    "mt_best","QE_mt_best","mt_best_from",
    "mt_best_DA_from","DA_mt_best",
    "literal_gt_ref"
]:
    df[c] = float("nan") if c.startswith(("QE_","DA_")) else ""

# ----- Generate & queue QE -----
print("[Info] Generating 3 translations per row and scoring QE …")
qe_items, qe_locs = [], []
for i, row in df.iterrows():
    src = row["src"]
    mts = translate_3_variants(src)
    df.at[i, "mt_1"], df.at[i, "mt_2"], df.at[i, "mt_3"] = mts

    for j, mt in enumerate(mts, start=1):
        if isinstance(mt, str) and mt.strip():
            qe_items.append({"src": src, "mt": mt})
            qe_locs.append((i, f"QE_mt_{j}"))

    if row["true_meaning"].strip():
        qe_items.append({"src": src, "mt": row["true_meaning"]})
        qe_locs.append((i, "QE_true"))
    if row["literal_translation"].strip():
        qe_items.append({"src": src, "mt": row["literal_translation"]})
        qe_locs.append((i, "QE_literal"))

# Batch QE
print(f"[Info] Total QE candidates to score: {len(qe_items)}")
with torch.no_grad():
    qe_scores = qe_model.predict(qe_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), score in zip(qe_locs, qe_scores):
    df.at[i, field] = float(score)

# ----- Pick best MT by QE -----
def _norm(v):
    try:
        v = float(v)
        return v if not math.isnan(v) else -math.inf
    except:
        return -math.inf

def _best_mt_row(r):
    cands = [("mt_1", r["mt_1"], r["QE_mt_1"]),
             ("mt_2", r["mt_2"], r["QE_mt_2"]),
             ("mt_3", r["mt_3"], r["QE_mt_3"])]
    return max(cands, key=lambda x: _norm(x[2]))

best_cols, best_texts, best_qes = [], [], []
for _, r in df.iterrows():
    col, txt, qe = _best_mt_row(r)
    best_cols.append(col); best_texts.append(txt); best_qes.append(qe)

df["mt_best"]      = best_texts
df["QE_mt_best"]   = best_qes
df["mt_best_from"] = best_cols

# ----- COMET-DA (ref = true_meaning) -----
da_items, da_locs = [], []
for i, r in df.iterrows():
    ref = r["true_meaning"]
    if not isinstance(ref, str) or not ref.strip():
        continue
    for label in ["mt_1","mt_2","mt_3","literal_translation","true_meaning"]:
        hyp = r[label] if label in r else r.get(label, "")
        if isinstance(hyp, str) and hyp.strip():
            da_items.append({"src": r["src"], "mt": hyp, "ref": ref})
            field = f"DA_{'literal' if label=='literal_translation' else ('true' if label=='true_meaning' else label)}"
            da_locs.append((i, field))

print(f"[Info] Total DA pairs to score: {len(da_items)}")
with torch.no_grad():
    da_scores = da_model.predict(da_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), s in zip(da_locs, da_scores):
    df.at[i, field] = float(s)

# Best-of-3 by DA among MT candidates
da_best_labels, da_best_scores = [], []
for _, r in df.iterrows():
    cands = [("mt_1", r.get("DA_mt_1", float("nan"))),
             ("mt_2", r.get("DA_mt_2", float("nan"))),
             ("mt_3", r.get("DA_mt_3", float("nan")))]
    cands = [(k, (v if isinstance(v,(int,float)) and not math.isnan(v) else -math.inf)) for k, v in cands]
    best_k, best_s = max(cands, key=lambda x: x[1])
    da_best_labels.append(best_k)
    da_best_scores.append(best_s if best_s != -math.inf else float("nan"))

df["mt_best_DA_from"] = da_best_labels
df["DA_mt_best"]      = da_best_scores

# ----- Summaries -----
def _nanmean(series):
    vals = pd.to_numeric(series, errors="coerce").dropna().tolist()
    return sum(vals)/len(vals) if vals else float("nan")

df["literal_gt_ref"] = (
    pd.to_numeric(df["QE_literal"], errors="coerce") >
    pd.to_numeric(df["QE_true"],    errors="coerce")
)

total_rows       = len(df)
avg_qe_best_mt   = _nanmean(df["QE_mt_best"])
avg_qe_true      = _nanmean(df["QE_true"])
avg_qe_literal   = _nanmean(df["QE_literal"])
num_best_gt_ref  = int((pd.to_numeric(df["QE_mt_best"], errors="coerce") >
                         pd.to_numeric(df["QE_true"],    errors="coerce")).sum())
num_lit_gt_ref   = int(df["literal_gt_ref"].fillna(False).sum())

avg_da_best_mt   = _nanmean(df["DA_mt_best"])
avg_da_true      = _nanmean(df["DA_true"])
avg_da_literal   = _nanmean(df["DA_literal"])

print("\n=== SUMMARY (COMET-QE) ===")
print(f"Samples                                        : {total_rows}")
print(f"Average QE (best of 3 MT)                      : {avg_qe_best_mt:.4f}")
print(f"Average QE (literal translation)               : {avg_qe_literal:.4f}")
print(f"Average QE (reference true_meaning)            : {avg_qe_true:.4f}")
print(f"Count where best-MT QE > true meaning QE       : {num_best_gt_ref} / {total_rows}")
print(f"Count where literal QE > true meaning QE       : {num_lit_gt_ref} / {total_rows}")

print("\n=== SUMMARY (COMET-DA; ref = true_meaning) ===")
print(f"Average DA (best of 3 MT)                      : {avg_da_best_mt:.4f}")
print(f"Average DA (true meaning)                      : {avg_da_true:.4f}")
print(f"Average DA (literal)                           : {avg_da_literal:.4f}")

# ----- Save outputs -----
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print(f"\nSaved CSV : {OUT_CSV}")
print(f"Saved JSON: {OUT_JSON}")


AssertionError: CSV not found: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-comet-da/splits/test.csv

# Evaluation on Evaluation of qwen2.5-3b-grpo-hybrid on 200 test.csv

In [ ]:
# === POST-TRAIN EVAL — Hybrid model (3 variants) : QE + DA + NLI + literal-overlap ===
import os, json, math, re, random
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer as HFTokenizer, AutoModelForSequenceClassification
from comet import download_model, load_from_checkpoint

# ----- Paths -----
CSV_PATH   = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid/splits/test.csv"  # test split saved during training
MODEL_PATH = "/content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid"                  # finetuned hybrid model dir
TOK_PATH   = "Qwen/Qwen2.5-3B-Instruct"                                                      # base tokenizer

# Outputs (into MODEL_PATH)
OUT_JSON = os.path.join(MODEL_PATH, "hybrid_eval_3mt.json")
OUT_CSV  = os.path.join(MODEL_PATH, "hybrid_eval_3mt.csv")

# ----- Config -----
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

MAX_NEW_TOKENS = 96
QE_CKPT = "Unbabel/wmt20-comet-qe-da"
DA_CKPT = "Unbabel/wmt22-comet-da"

# NLI config
NLI_CKPT = "roberta-large-mnli"
ENTAILMENT_ID = 2  # roberta-mnli order: contradiction/neutral/entailment

# ----- Load data -----
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

required = {"src", "true_meaning"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV must have columns {sorted(required)}. Missing: {sorted(missing)}")

if "literal_translation" not in df.columns:
    df["literal_translation"] = ""
for c in ["src","true_meaning","literal_translation"]:
    df[c] = df[c].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)
print(f"[Info] Loaded {len(df)} rows from {CSV_PATH}")

# ----- Load model/tokenizer -----
print(f"[Info] Loading tokenizer: {TOK_PATH}")
tok = AutoTokenizer.from_pretrained(TOK_PATH, trust_remote_code=True, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
EOS_ID = tok.eos_token_id

print(f"[Info] Loading FINETUNED model: {MODEL_PATH} on {DEVICE}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
).eval()

# ----- Simple prompt (match training: convey TRUE meaning if figurative) -----
SYSTEM_MSG = (
    "Translate Hindi to natural English. If the text is figurative or idiomatic, convey the intended TRUE meaning. "
    "Output ONE English line only."
)

def build_prompt(src_text: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user",   "content": f"Hindi: {src_text}\nEnglish:"}
    ]
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.no_grad()
def translate_once(text: str, *, do_sample: bool, temperature: float = 0.7, top_p: float = 0.9):
    enc = tok(build_prompt(text), return_tensors="pt").to(next(model.parameters()).device)
    input_len = enc.input_ids.shape[1]
    gen_kwargs = dict(max_new_tokens=MAX_NEW_TOKENS, pad_token_id=tok.pad_token_id, eos_token_id=EOS_ID)
    if do_sample:
        gen_kwargs.update(dict(do_sample=True, temperature=float(temperature), top_p=float(top_p), repetition_penalty=1.05))
    else:
        gen_kwargs.update(dict(do_sample=False))
    out = model.generate(**enc, **gen_kwargs)
    cont = tok.decode(out[0, input_len:], skip_special_tokens=True).strip()
    for p in ("English:", "english:", "Translation:", "translation:"):
        if cont.startswith(p): cont = cont[len(p):].strip()
    return " ".join(cont.split())[:140]

def translate_3_variants(text: str):
    # 1 greedy + 2 sampled
    return [
        translate_once(text, do_sample=False),
        translate_once(text, do_sample=True,  temperature=0.7, top_p=0.9),
        translate_once(text, do_sample=True,  temperature=1.0, top_p=0.92),
    ]

# ----- Load scorers (QE, DA, NLI) -----
print("[Info] Loading COMET QE …")
qe_model = load_from_checkpoint(download_model(QE_CKPT)).eval()
print("[Info] Loading COMET DA …")
da_model = load_from_checkpoint(download_model(DA_CKPT)).eval()

print("[Info] Loading NLI …")
nli_tok = HFTokenizer.from_pretrained(NLI_CKPT)
nli_mod = AutoModelForSequenceClassification.from_pretrained(NLI_CKPT).to(DEVICE).eval()

@torch.no_grad()
def nli_entail_prob(premise: str, hypothesis: str) -> float:
    if not premise or not hypothesis: return 0.0
    enc = nli_tok(premise, hypothesis, return_tensors="pt", truncation=True, padding=True).to(DEVICE)
    logits = nli_mod(**enc).logits.float()
    return float(logits.softmax(dim=-1)[0, ENTAILMENT_ID].item())

def literal_overlap(mt: str, lit: str) -> float:
    if not isinstance(mt,str) or not isinstance(lit,str): return 0.0
    A = set(re.findall(r"[a-z]+", mt.lower()))
    B = set(re.findall(r"[a-z]+", lit.lower()))
    if not A or not B: return 0.0
    return len(A & B) / max(1, len(A | B))

# ----- Prepare output columns -----
for c in [
    "mt_1","mt_2","mt_3",
    "QE_mt_1","QE_mt_2","QE_mt_3","QE_true","QE_literal",
    "DA_mt_1","DA_mt_2","DA_mt_3","DA_true","DA_literal",
    "NLI_mt_1","NLI_mt_2","NLI_mt_3","NLI_true","NLI_literal",
    "LIT_mt_1","LIT_mt_2","LIT_mt_3",
    "mt_best_by_QE","QE_mt_best","mt_best_from",
    "mt_best_by_DA","DA_mt_best",
]:
    df[c] = float("nan") if c.startswith(("QE_","DA_","NLI_","LIT_")) else ""

# ----- Generate translations -----
print("[Info] Generating 3 translations per row …")
for i, row in df.iterrows():
    mts = translate_3_variants(row["src"])
    df.at[i,"mt_1"], df.at[i,"mt_2"], df.at[i,"mt_3"] = mts

# ----- Batch COMET-QE -----
qe_items, qe_locs = [], []
for i, r in df.iterrows():
    src = r["src"]
    for j in (1,2,3):
        mt = r[f"mt_{j}"]
        if isinstance(mt,str) and mt.strip():
            qe_items.append({"src":src, "mt":mt}); qe_locs.append((i,f"QE_mt_{j}"))
    if r["true_meaning"].strip():
        qe_items.append({"src":src, "mt":r["true_meaning"]}); qe_locs.append((i,"QE_true"))
    if r["literal_translation"].strip():
        qe_items.append({"src":src, "mt":r["literal_translation"]}); qe_locs.append((i,"QE_literal"))

print(f"[Info] Total QE candidates: {len(qe_items)}")
with torch.no_grad():
    qe_scores = qe_model.predict(qe_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), s in zip(qe_locs, qe_scores):
    df.at[i, field] = float(s)

# ----- Pick best by QE among MTs -----
def _norm(v):
    try:
        v = float(v)
        return v if not math.isnan(v) else -math.inf
    except:
        return -math.inf

best_qe_cols, best_qe_texts, best_qe_vals = [], [], []
for _, r in df.iterrows():
    cands = [("mt_1", r["mt_1"], r["QE_mt_1"]),
             ("mt_2", r["mt_2"], r["QE_mt_2"]),
             ("mt_3", r["mt_3"], r["QE_mt_3"])]
    col, txt, qe = max(cands, key=lambda x: _norm(x[2]))
    best_qe_cols.append(col); best_qe_texts.append(txt); best_qe_vals.append(qe)

df["mt_best_by_QE"] = best_qe_texts
df["QE_mt_best"]    = best_qe_vals
df["mt_best_from"]  = best_qe_cols

# ----- Batch COMET-DA (ref = true_meaning) -----
da_items, da_locs = [], []
for i, r in df.iterrows():
    ref = r["true_meaning"]
    if not isinstance(ref,str) or not ref.strip(): continue
    src = r["src"]
    for label in ["mt_1","mt_2","mt_3","literal_translation","true_meaning"]:
        hyp = r.get(label, "")
        if isinstance(hyp,str) and hyp.strip():
            da_items.append({"src":src, "mt":hyp, "ref":ref})
            field = f"DA_{'literal' if label=='literal_translation' else ('true' if label=='true_meaning' else label)}"
            da_locs.append((i, field))

print(f"[Info] Total DA pairs: {len(da_items)}")
with torch.no_grad():
    da_scores = da_model.predict(da_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), s in zip(da_locs, da_scores):
    df.at[i, field] = float(s)

# ----- Best-of-3 by DA -----
da_best_labels, da_best_scores = [], []
for _, r in df.iterrows():
    cands = [("mt_1", r.get("DA_mt_1", float("nan"))),
             ("mt_2", r.get("DA_mt_2", float("nan"))),
             ("mt_3", r.get("DA_mt_3", float("nan")))]
    cands = [(k, (v if isinstance(v,(int,float)) and not math.isnan(v) else -math.inf)) for k, v in cands]
    best_k, best_s = max(cands, key=lambda x: x[1])
    da_best_labels.append(best_k)
    da_best_scores.append(best_s if best_s != -math.inf else float("nan"))

df["mt_best_by_DA"] = da_best_labels
df["DA_mt_best"]    = da_best_scores

# ----- NLI entailment & literal-overlap (for MTs + refs) -----
@torch.no_grad()
def safe_entail(mt, ref):
    try:
        return nli_entail_prob(mt, ref) if (isinstance(mt,str) and mt.strip() and isinstance(ref,str) and ref.strip()) else float("nan")
    except Exception:
        return float("nan")

def lit_overlap(mt, lit):
    try:
        return literal_overlap(mt, lit) if (isinstance(mt,str) and mt.strip() and isinstance(lit,str) and lit.strip()) else float("nan")
    except Exception:
        return float("nan")

for i, r in df.iterrows():
    ref, lit = r["true_meaning"], r["literal_translation"]
    for j in (1,2,3):
        mt = r[f"mt_{j}"]
        df.at[i, f"NLI_mt_{j}"] = safe_entail(mt, ref)
        df.at[i, f"LIT_mt_{j}"] = lit_overlap(mt, lit)
    # (Optional) NLI for references themselves (sanity)
    df.at[i, "NLI_true"]    = safe_entail(ref, ref)
    df.at[i, "NLI_literal"] = safe_entail(lit, ref) if isinstance(lit,str) and lit.strip() else float("nan")

# ----- Summaries -----
def _nanmean(series):
    vals = pd.to_numeric(series, errors="coerce").dropna().tolist()
    return sum(vals)/len(vals) if vals else float("nan")

total_rows = len(df)

avg_qe_best_mt  = _nanmean(df["QE_mt_best"])
avg_qe_true     = _nanmean(df["QE_true"])
avg_qe_literal  = _nanmean(df["QE_literal"])
num_best_gt_ref = int((pd.to_numeric(df["QE_mt_best"], errors="coerce") >
                       pd.to_numeric(df["QE_true"],    errors="coerce")).sum())

avg_da_best_mt  = _nanmean(df["DA_mt_best"])
avg_da_true     = _nanmean(df["DA_true"])
avg_da_literal  = _nanmean(df["DA_literal"])

# NLI & literal overlap summaries on MTs (mean over mt_1..mt_3)
avg_nli_mt = _nanmean(pd.concat([df["NLI_mt_1"], df["NLI_mt_2"], df["NLI_mt_3"]], ignore_index=True))
avg_lit_mt = _nanmean(pd.concat([df["LIT_mt_1"], df["LIT_mt_2"], df["LIT_mt_3"]], ignore_index=True))

print("\n=== SUMMARY (COMET-QE) ===")
print(f"Samples                                 : {total_rows}")
print(f"Average QE (best-of-3 MT)               : {avg_qe_best_mt:.4f}")
print(f"Average QE (reference true_meaning)     : {avg_qe_true:.4f}")
print(f"Average QE (literal translation)        : {avg_qe_literal:.4f}")
print(f"Count best-MT QE > reference QE         : {num_best_gt_ref} / {total_rows}")

print("\n=== SUMMARY (COMET-DA; ref = true_meaning) ===")
print(f"Average DA (best-of-3 MT)               : {avg_da_best_mt:.4f}")
print(f"Average DA (true meaning)               : {avg_da_true:.4f}")
print(f"Average DA (literal)                    : {avg_da_literal:.4f}")

print("\n=== SUMMARY (NLI & Literal-Overlap) ===")
print(f"Average NLI entailment (MT ⇒ ref)       : {avg_nli_mt:.4f}")
print(f"Average literal-overlap Jaccard (MT,lit): {avg_lit_mt:.4f}  (lower is better)")

# ----- Save outputs -----
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print(f"\nSaved CSV : {OUT_CSV}")
print(f"Saved JSON: {OUT_JSON}")



[Info] Loaded 200 rows from /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid/splits/test.csv
[Info] Loading tokenizer: Qwen/Qwen2.5-3B-Instruct
[Info] Loading FINETUNED model: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid on cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[Info] Loading COMET QE …


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt20-comet-qe-da/snapshots/2e7ffc84fb67d99cf92506611766463bb9230cfb/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


...

[Info] Generating 3 translations per row …


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Info] Total QE candidates: 1000


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Info] Total DA pairs: 1000

=== SUMMARY (COMET-QE) ===
Samples                                 : 200
Average QE (best-of-3 MT)               : -0.1205
Average QE (reference true_meaning)     : -0.4446
Average QE (literal translation)        : -0.2685
Count best-MT QE > reference QE         : 155 / 200

=== SUMMARY (COMET-DA; ref = true_meaning) ===
Average DA (best-of-3 MT)               : 0.4867
Average DA (true meaning)               : 0.9748
Average DA (literal)                    : 0.5412

=== SUMMARY (NLI & Literal-Overlap) ===
Average NLI entailment (MT ⇒ ref)       : 0.3254
Average literal-overlap Jaccard (MT,lit): 0.1475  (lower is better)

Saved CSV : /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid/hybrid_eval_3mt.csv
Saved JSON: /content/drive/MyDrive/GRPO-Hindi-Eng/qwen2.5-3b-grpo-hybrid/hybrid_eval_3mt.json


# Evaluation of LoRA SFT + GRPO Training on 200.csv

In [ ]:
# EVAL: PEFT (LoRA) model on test.csv with QE+DA (greedy + 2 sampled)
import os, json, math
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----- Paths (EDIT THESE) -----
ADAPTERS_DIR = "/content/drive/MyDrive/newfolder/qwen2.5-3b-grpo-comet-da-fast_800-200"  # your trained LoRA save dir
CSV_PATH     = os.path.join(ADAPTERS_DIR, "splits/test.csv")                              # test split saved during training
BASE_MODEL   = "Qwen/Qwen2.5-3B-Instruct"                                                 # base
TOK_PATH     = "Qwen/Qwen2.5-3B-Instruct"

OUT_DIR  = os.path.join(ADAPTERS_DIR, "eval_qe_da_3mt")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_JSON = os.path.join(OUT_DIR, "comet_results_3mt_FINETUNED.json")
OUT_CSV  = os.path.join(OUT_DIR, "comet_results_3mt_FINETUNED.csv")

# ----- Config -----
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

MAX_NEW_TOKENS = 48
QE_CKPT = "Unbabel/wmt20-comet-qe-da"
DA_CKPT = "Unbabel/wmt22-comet-da"

MERGE_ADAPTERS_FOR_EVAL = True   # merges LoRA into base weights for faster generation

# ----- Load data -----
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

required = {"src", "true_meaning"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV must have columns {sorted(required)}. Missing: {sorted(missing)}")

if "literal_translation" not in df.columns:
    df["literal_translation"] = ""
for col in ["src", "true_meaning", "literal_translation"]:
    df[col] = df[col].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)
print(f"[Info] Loaded {len(df)} rows from {CSV_PATH}")

# ----- Load tokenizer -----
print(f"[Info] Loading tokenizer: {TOK_PATH}")
tok = AutoTokenizer.from_pretrained(TOK_PATH, trust_remote_code=True, use_fast=True)
tok.padding_side = "left"               # decoder-only models need left padding for correct generation
if tok.pad_token_id is None:
    tok.pad_token_id = tok.eos_token_id
EOS_ID = tok.eos_token_id

# ----- Load base + adapters (PEFT) -----
print(f"[Info] Loading BASE model: {BASE_MODEL}")
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, device_map="auto", dtype=DTYPE, trust_remote_code=True
)

from peft import PeftModel, AutoPeftModelForCausalLM
try:
    # If adapters directory contains full PEFT config, this loads directly:
    model = AutoPeftModelForCausalLM.from_pretrained(
        ADAPTERS_DIR, device_map="auto", dtype=DTYPE, trust_remote_code=True
    ).eval()
except Exception:
    # Fallback: attach adapters to base
    model = PeftModel.from_pretrained(base, ADAPTERS_DIR).eval()

# Optionally merge LoRA weights for faster eval
if MERGE_ADAPTERS_FOR_EVAL and hasattr(model, "merge_and_unload"):
    print("[Info] Merging LoRA adapters into base weights for evaluation …")
    model = model.merge_and_unload()
model.config.use_cache = True  # speed up autoregressive generation

SYSTEM_MSG = "You are a professional translator. Translate Hindi to natural English. Keep the meaning and tone."
def _prompt(text: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user",   "content": f"Hindi: {text}\nEnglish:"}
    ]
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.no_grad()
def translate_once(text: str, *, do_sample: bool, temperature: float = 1.0, top_p: float = 0.9):
    enc = tok(_prompt(text), return_tensors="pt", padding=True, truncation=True).to(next(model.parameters()).device)
    input_len = enc.input_ids.shape[1]
    gen_kwargs = dict(max_new_tokens=MAX_NEW_TOKENS, pad_token_id=tok.pad_token_id, eos_token_id=EOS_ID)
    if do_sample:
        gen_kwargs.update(dict(do_sample=True, temperature=temperature, top_p=top_p))
    else:
        gen_kwargs.update(dict(do_sample=False))
    out = model.generate(**enc, **gen_kwargs)
    cont = tok.decode(out[0, input_len:], skip_special_tokens=True).strip()
    if cont.lower().startswith("english:"):
        cont = cont[len("english:"):].strip()
    return cont

def translate_3_variants(text: str):
    # 1 greedy + 2 sampled (same pattern as your baseline)
    return [
        translate_once(text, do_sample=False),
        translate_once(text, do_sample=True,  temperature=0.7, top_p=0.9),
        translate_once(text, do_sample=True,  temperature=1.0, top_p=0.92),
    ]

# ----- Load COMET models (QE + DA) -----
print("[Info] Loading COMET QE …")
qe_model = load_from_checkpoint(download_model(QE_CKPT)).eval()
try:
    if DEVICE == "cuda": qe_model.model.to("cuda")
except Exception:
    pass

print("[Info] Loading COMET DA …")
da_model = load_from_checkpoint(download_model(DA_CKPT)).eval()
try:
    if DEVICE == "cuda": da_model.model.to("cuda")
except Exception:
    pass

# ----- Prepare output columns -----
for c in [
    "mt_1","mt_2","mt_3",
    "QE_mt_1","QE_mt_2","QE_mt_3","QE_true","QE_literal","QE_dataset_mt",
    "DA_mt_1","DA_mt_2","DA_mt_3","DA_true","DA_literal",
    "mt_best","QE_mt_best","mt_best_from",
    "mt_best_DA_from","DA_mt_best",
    "literal_gt_ref"
]:
    df[c] = float("nan") if c.startswith(("QE_","DA_")) else ""

# If your CSV has a model baseline translation column, score it too:
if "mt" not in df.columns:
    df["mt"] = ""  # optional dataset MT column

# ----- Generate & queue QE -----
print("[Info] Generating 3 translations per row and scoring QE …")
qe_items, qe_locs = [], []
for i, row in df.iterrows():
    src = row["src"]
    mts = translate_3_variants(src)
    df.at[i, "mt_1"], df.at[i, "mt_2"], df.at[i, "mt_3"] = mts

    for j, mt in enumerate(mts, start=1):
        if isinstance(mt, str) and mt.strip():
            qe_items.append({"src": src, "mt": mt})
            qe_locs.append((i, f"QE_mt_{j}"))

    if row["true_meaning"].strip():
        qe_items.append({"src": src, "mt": row["true_meaning"]})
        qe_locs.append((i, "QE_true"))
    if row["literal_translation"].strip():
        qe_items.append({"src": src, "mt": row["literal_translation"]})
        qe_locs.append((i, "QE_literal"))
    if row["mt"].strip():  # optional dataset MT column
        qe_items.append({"src": src, "mt": row["mt"]})
        qe_locs.append((i, "QE_dataset_mt"))

# Batch QE
print(f"[Info] Total QE candidates to score: {len(qe_items)}")
with torch.no_grad():
    qe_scores = qe_model.predict(qe_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), score in zip(qe_locs, qe_scores):
    df.at[i, field] = float(score)

# ----- Pick best MT by QE -----
def _norm(v):
    try:
        v = float(v)
        return v if not math.isnan(v) else -math.inf
    except:
        return -math.inf

def _best_mt_row(r):
    cands = [("mt_1", r["mt_1"], r["QE_mt_1"]),
             ("mt_2", r["mt_2"], r["QE_mt_2"]),
             ("mt_3", r["mt_3"], r["QE_mt_3"])]
    return max(cands, key=lambda x: _norm(x[2]))

best_cols, best_texts, best_qes = [], [], []
for _, r in df.iterrows():
    col, txt, qe = _best_mt_row(r)
    best_cols.append(col); best_texts.append(txt); best_qes.append(qe)

df["mt_best"]      = best_texts
df["QE_mt_best"]   = best_qes
df["mt_best_from"] = best_cols

# ----- COMET-DA (ref = true_meaning) -----
da_items, da_locs = [], []
for i, r in df.iterrows():
    ref = r["true_meaning"]
    if not isinstance(ref, str) or not ref.strip():
        continue
    for label in ["mt_1","mt_2","mt_3","literal_translation","true_meaning"]:
        hyp = r[label] if label in r else r.get(label, "")
        if isinstance(hyp, str) and hyp.strip():
            da_items.append({"src": r["src"], "mt": hyp, "ref": ref})
            field = f"DA_{'literal' if label=='literal_translation' else ('true' if label=='true_meaning' else label)}"
            da_locs.append((i, field))

print(f"[Info] Total DA pairs to score: {len(da_items)}")
with torch.no_grad():
    da_scores = da_model.predict(da_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), s in zip(da_locs, da_scores):
    df.at[i, field] = float(s)

# Best-of-3 by DA among MT candidates
da_best_labels, da_best_scores = [], []
for _, r in df.iterrows():
    cands = [("mt_1", r.get("DA_mt_1", float("nan"))),
             ("mt_2", r.get("DA_mt_2", float("nan"))),
             ("mt_3", r.get("DA_mt_3", float("nan")))]
    cands = [(k, (v if isinstance(v,(int,float)) and not math.isnan(v) else -math.inf)) for k, v in cands]
    best_k, best_s = max(cands, key=lambda x: x[1])
    da_best_labels.append(best_k)
    da_best_scores.append(best_s if best_s != -math.inf else float("nan"))

df["mt_best_DA_from"] = da_best_labels
df["DA_mt_best"]      = da_best_scores

# ----- Summaries -----
def _nanmean(series):
    vals = pd.to_numeric(series, errors="coerce").dropna().tolist()
    return sum(vals)/len(vals) if vals else float("nan")

df["literal_gt_ref"] = (
    pd.to_numeric(df["QE_literal"], errors="coerce") >
    pd.to_numeric(df["QE_true"],    errors="coerce")
)

total_rows       = len(df)
avg_qe_best_mt   = _nanmean(df["QE_mt_best"])
avg_qe_true      = _nanmean(df["QE_true"])
avg_qe_literal   = _nanmean(df["QE_literal"])
num_best_gt_ref  = int((pd.to_numeric(df["QE_mt_best"], errors="coerce") >
                         pd.to_numeric(df["QE_true"],    errors="coerce")).sum())
num_lit_gt_ref   = int(df["literal_gt_ref"].fillna(False).sum())

avg_da_best_mt   = _nanmean(df["DA_mt_best"])
avg_da_true      = _nanmean(df["DA_true"])
avg_da_literal   = _nanmean(df["DA_literal"])

print("\n=== SUMMARY (COMET-QE) ===")
print(f"Samples                                        : {total_rows}")
print(f"Average QE (best of 3 MT)                      : {avg_qe_best_mt:.4f}")
print(f"Average QE (literal translation)               : {avg_qe_literal:.4f}")
print(f"Average QE (reference true_meaning)            : {avg_qe_true:.4f}")
print(f"Count where best-MT QE > true meaning QE       : {num_best_gt_ref} / {total_rows}")
print(f"Count where literal QE > true meaning QE       : {num_lit_gt_ref} / {total_rows}")

print("\n=== SUMMARY (COMET-DA; ref = true_meaning) ===")
print(f"Average DA (best of 3 MT)                      : {avg_da_best_mt:.4f}")
print(f"Average DA (true meaning)                      : {avg_da_true:.4f}")
print(f"Average DA (literal)                           : {avg_da_literal:.4f}")

# ----- Save outputs -----
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print(f"\nSaved CSV : {OUT_CSV}")
print(f"Saved JSON: {OUT_JSON}")


[Info] Loaded 200 rows from /content/drive/MyDrive/newfolder/qwen2.5-3b-grpo-comet-da-fast_800-200/splits/test.csv
[Info] Loading tokenizer: Qwen/Qwen2.5-3B-Instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

[Info] Generating 3 translations per row and scoring QE …


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Info] Total QE candidates to score: 1000


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[Info] Total DA pairs to score: 1000

=== SUMMARY (COMET-QE) ===
Samples                                        : 200
Average QE (best of 3 MT)                      : -0.1236
Average QE (literal translation)               : -0.2683
Average QE (reference true_meaning)            : -0.4447
Count where best-MT QE > true meaning QE       : 171 / 200
Count where literal QE > true meaning QE       : 145 / 200

=== SUMMARY (COMET-DA; ref = true_meaning) ===
Average DA (best of 3 MT)                      : 0.5443
Average DA (true meaning)                      : 0.9748
Average DA (literal)                           : 0.5412

Saved CSV : /content/drive/MyDrive/newfolder/qwen2.5-3b-grpo-comet-da-fast_800-200/eval_qe_da_3mt/comet_results_3mt_FINETUNED.csv
Saved JSON: /content/drive/MyDrive/newfolder/qwen2.5-3b-grpo-comet-da-fast_800-200/eval_qe_da_3mt/comet_results_3mt_FINETUNED.json


# Evaluation of COMET-DA as Reward with Llama 3.2-3B-Instruct as the baseline model

In [5]:
# === GRPO POST-TRAIN EVAL (3 variants, QE+DA, baseline-matching) — Llama COMET-DA finetune ===
import os, json, math, re, random
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----- Paths -----
# Use the test split saved during training (your llama COMET-DA run)
CSV_PATH    = "/content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-da/splits/test.csv"
MODEL_PATH  = "/content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-da"  # fine-tuned weights
# Prefer using the tokenizer saved with the model to avoid HF gating:
TOK_PATH    = MODEL_PATH  # or: "meta-llama/Llama-3.2-3B-Instruct" (requires HF access)

# Output filenames (saved next to MODEL_PATH)
OUT_JSON = os.path.join(MODEL_PATH, "comet_results_3mt_FINETUNED_DA.json")
OUT_CSV  = os.path.join(MODEL_PATH, "comet_results_3mt_FINETUNED_DA.csv")

# ----- Config -----
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

MAX_NEW_TOKENS = 96
QE_CKPT = "Unbabel/wmt20-comet-qe-da"
DA_CKPT = "Unbabel/wmt22-comet-da"

# ----- Load data -----
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

required = {"src", "true_meaning"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV must have columns {sorted(required)}. Missing: {sorted(missing)}")

if "literal_translation" not in df.columns:
    df["literal_translation"] = ""

for col in ["src", "true_meaning", "literal_translation"]:
    df[col] = df[col].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)
print(f"[Info] Loaded {len(df)} rows from {CSV_PATH}")

# ----- Load model/tokenizer -----
print(f"[Info] Loading tokenizer from: {TOK_PATH}")
tok = AutoTokenizer.from_pretrained(TOK_PATH, trust_remote_code=True, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
EOS_ID = tok.eos_token_id

print(f"[Info] Loading fine-tuned model: {MODEL_PATH} on {DEVICE}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
).eval()

# ----- Few-shot prompt (matches training; forces English-only output) -----
FEW_SHOTS = [
    ("उसकी बातें मेरे सर के ऊपर से गईं।", "I couldn’t make heads or tails of it."),
    ("वह टस से मस नहीं हुआ।",            "He wouldn’t budge an inch."),
    ("तेल देखो और तेल की धार देखो।",     "Let’s wait and see how it plays out."),
]
SYSTEM_MSG = (
    "You are an expert Hindi→English translator for IDIOMS. "
    "Convey the FIGURATIVE meaning; avoid literal renderings."
)
USER_INSTR = (
    "Task: Translate the Hindi idiom into natural English conveying the figurative sense.\n"
    "Rules:\n"
    "1) Avoid literal/word-for-word translations.\n"
    "2) Output ONLY the English translation (no labels, no quotes).\n"
    "3) Prefer common idiomatic English; otherwise paraphrase plainly.\n"
    "4) ≤ 20 words; natural everyday tone."
)

def build_prompt(tok, src_text: str, k_shots: int = 2) -> str:
    shots = random.sample(FEW_SHOTS, k=min(k_shots, len(FEW_SHOTS)))
    messages = [{"role": "system", "content": SYSTEM_MSG}]
    for hi, en in shots:
        messages.append({"role": "user", "content": f"{USER_INSTR}\n\nHindi: {hi}\nEnglish:"})
        messages.append({"role": "assistant", "content": en})
    messages.append({"role": "user", "content": f"{USER_INSTR}\n\nHindi: {src_text}\nEnglish:"})
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

_EN_LINE = re.compile(r"^[^\n\r]*", flags=re.IGNORECASE)
def clean_english(text: str) -> str:
    line = _EN_LINE.findall(text.strip())[0] if text.strip() else ""
    if line.lower().startswith("english:"): line = line[len("english:"):].strip()
    return re.sub(r'\s+', ' ', line.strip().strip('"').strip("'").strip("`")).strip()

def _prompt(text: str) -> str:
    return build_prompt(tok, text, k_shots=2)

@torch.no_grad()
def translate_once(text: str, *, do_sample: bool, temperature: float = 1.0, top_p: float = 0.9):
    enc = tok(_prompt(text), return_tensors="pt").to(next(model.parameters()).device)
    input_len = enc.input_ids.shape[1]
    gen_kwargs = dict(max_new_tokens=MAX_NEW_TOKENS, pad_token_id=tok.pad_token_id, eos_token_id=EOS_ID)
    if do_sample:
        gen_kwargs.update(dict(do_sample=True, temperature=float(temperature), top_p=float(top_p)))
    else:
        gen_kwargs.update(dict(do_sample=False))
    out = model.generate(**enc, **gen_kwargs)
    cont = tok.decode(out[0, input_len:], skip_special_tokens=True)
    return clean_english(cont)

def translate_3_variants(text: str):
    return [
        translate_once(text, do_sample=False),
        translate_once(text, do_sample=True,  temperature=0.7, top_p=0.9),
        translate_once(text, do_sample=True,  temperature=1.0, top_p=0.92),
    ]

# ----- Load COMET models (QE + DA) -----
print("[Info] Loading COMET QE …")
qe_model = load_from_checkpoint(download_model(QE_CKPT)).eval()
print("[Info] Loading COMET DA …")
da_model = load_from_checkpoint(download_model(DA_CKPT)).eval()

# ----- Prepare output columns -----
for c in [
    "mt_1","mt_2","mt_3",
    "QE_mt_1","QE_mt_2","QE_mt_3","QE_true","QE_literal","QE_dataset_mt",
    "DA_mt_1","DA_mt_2","DA_mt_3","DA_true","DA_literal",
    "mt_best","QE_mt_best","mt_best_from",
    "mt_best_DA_from","DA_mt_best",
    "literal_gt_ref"
]:
    df[c] = float("nan") if c.startswith(("QE_","DA_")) else ""

# ----- Generate & QE -----
qe_items, qe_locs = [], []
for i, row in df.iterrows():
    src = row["src"]
    mts = translate_3_variants(src)
    df.at[i, "mt_1"], df.at[i, "mt_2"], df.at[i, "mt_3"] = mts
    for j, mt in enumerate(mts, start=1):
        if isinstance(mt, str) and mt.strip():
            qe_items.append({"src": src, "mt": mt})
            qe_locs.append((i, f"QE_mt_{j}"))
    if row["true_meaning"].strip():
        qe_items.append({"src": src, "mt": row["true_meaning"]}); qe_locs.append((i, "QE_true"))
    if row["literal_translation"].strip():
        qe_items.append({"src": src, "mt": row["literal_translation"]}); qe_locs.append((i, "QE_literal"))

with torch.no_grad():
    qe_scores = qe_model.predict(qe_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), score in zip(qe_locs, qe_scores):
    df.at[i, field] = float(score)

# ----- Pick QE-best -----
def _norm(v):
    try: v = float(v); return v if not math.isnan(v) else -math.inf
    except: return -math.inf

best_cols, best_texts, best_qes = [], [], []
for _, r in df.iterrows():
    cands = [("mt_1", r["mt_1"], r["QE_mt_1"]),
             ("mt_2", r["mt_2"], r["QE_mt_2"]),
             ("mt_3", r["mt_3"], r["QE_mt_3"])]
    col, txt, qe = max(cands, key=lambda x: _norm(x[2]))
    best_cols.append(col); best_texts.append(txt); best_qes.append(qe)

df["mt_best"]      = best_texts
df["QE_mt_best"]   = best_qes
df["mt_best_from"] = best_cols

# ----- COMET-DA (ref = true_meaning) -----
da_items, da_locs = [], []
for i, r in df.iterrows():
    ref = r["true_meaning"]
    if not isinstance(ref, str) or not ref.strip(): continue
    for label in ["mt_1","mt_2","mt_3","literal_translation","true_meaning"]:
        hyp = r[label] if label in r else r.get(label, "")
        if isinstance(hyp, str) and hyp.strip():
            da_items.append({"src": r["src"], "mt": hyp, "ref": ref})
            field = f"DA_{'literal' if label=='literal_translation' else ('true' if label=='true_meaning' else label)}"
            da_locs.append((i, field))

with torch.no_grad():
    da_scores = da_model.predict(da_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), s in zip(da_locs, da_scores):
    df.at[i, field] = float(s)

# Best-of-3 by DA among MT candidates
da_best_labels, da_best_scores = [], []
for _, r in df.iterrows():
    cands = [("mt_1", r.get("DA_mt_1", float("nan"))),
             ("mt_2", r.get("DA_mt_2", float("nan"))),
             ("mt_3", r.get("DA_mt_3", float("nan")))]
    cands = [(k, (v if isinstance(v,(int,float)) and not math.isnan(v) else -math.inf)) for k, v in cands]
    best_k, best_s = max(cands, key=lambda x: x[1])
    da_best_labels.append(best_k); da_best_scores.append(best_s if best_s!=-math.inf else float("nan"))

df["mt_best_DA_from"] = da_best_labels
df["DA_mt_best"]      = da_best_scores

# ----- Summaries -----
def _nanmean(s):
    vals = pd.to_numeric(s, errors="coerce").dropna().tolist()
    return sum(vals)/len(vals) if vals else float("nan")

df["literal_gt_ref"] = (
    pd.to_numeric(df["QE_literal"], errors="coerce") >
    pd.to_numeric(df["QE_true"],    errors="coerce")
)

total_rows       = len(df)
avg_qe_best_mt   = _nanmean(df["QE_mt_best"])
avg_qe_true      = _nanmean(df["QE_true"])
avg_qe_literal   = _nanmean(df["QE_literal"])
num_best_gt_ref  = int((pd.to_numeric(df["QE_mt_best"], errors="coerce") >
                         pd.to_numeric(df["QE_true"],    errors="coerce")).sum())
num_lit_gt_ref   = int(df["literal_gt_ref"].fillna(False).sum())

avg_da_best_mt   = _nanmean(df["DA_mt_best"])
avg_da_true      = _nanmean(df["DA_true"])
avg_da_literal   = _nanmean(df["DA_literal"])

print("\n=== SUMMARY (COMET-QE) — Llama COMET-DA finetune ===")
print(f"Samples                              : {total_rows}")
print(f"Average QE (best of 3 MT)            : {avg_qe_best_mt:.4f}")
print(f"Average QE (literal translation)     : {avg_qe_literal:.4f}")
print(f"Average QE (reference true_meaning)  : {avg_qe_true:.4f}")
print(f"Best-MT QE > true QE                 : {num_best_gt_ref} / {total_rows}")
print(f"Literal QE > true QE                 : {num_lit_gt_ref} / {total_rows}")

print("\n=== SUMMARY (COMET-DA; ref = true_meaning) — Llama COMET-DA finetune ===")
print(f"Average DA (best of 3 MT)            : {avg_da_best_mt:.4f}")
print(f"Average DA (true vs ref)             : {avg_da_true:.4f}")
print(f"Average DA (literal vs ref)          : {avg_da_literal:.4f}")

# ----- Save outputs -----
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print(f"\nSaved CSV : {OUT_CSV}")
print(f"Saved JSON: {OUT_JSON}")


[Info] Loaded 500 rows from /content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-da/splits/test.csv
[Info] Loading tokenizer from: /content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-da
[Info] Loading fine-tuned model: /content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-da on cuda


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[Info] Loading COMET QE …


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

checkpoints/model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.12/dist-packages/torch/__init__.py:155


=== SUMMARY (COMET-QE) — Llama COMET-DA finetune ===
Samples                              : 500
Average QE (best of 3 MT)            : -0.1690
Average QE (literal translation)     : -0.2853
Average QE (reference true_meaning)  : -0.4293
Best-MT QE > true QE                 : 416 / 500
Literal QE > true QE                 : 345 / 500

=== SUMMARY (COMET-DA; ref = true_meaning) — Llama COMET-DA finetune ===
Average DA (best of 3 MT)            : 0.5336
Average DA (true vs ref)             : 0.9745
Average DA (literal vs ref)          : 0.5476

Saved CSV : /content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-da/comet_results_3mt_FINETUNED_DA.csv
Saved JSON: /content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-da/comet_results_3mt_FINETUNED_DA.json


# Evaluation of COMET-QE as Reward with Llama 3.2-3B-Instruct as the baseline model

In [ ]:
# === GRPO POST-TRAIN EVAL (3 variants, QE+DA, baseline-matching) — Llama COMET-QE finetune ===
import os, json, math, re, random
from pathlib import Path
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint

# ----- Paths -----
# Use the test split saved during your Llama COMET-QE training
CSV_PATH    = "/content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-qe/splits/test.csv"
MODEL_PATH  = "/content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-qe"  # fine-tuned weights
TOK_PATH    = MODEL_PATH  # avoids HF gating; or "meta-llama/Llama-3.2-3B-Instruct" if you have HF access

# Output filenames
OUT_JSON = os.path.join(MODEL_PATH, "comet_results_3mt_FINETUNED_QE.json")
OUT_CSV  = os.path.join(MODEL_PATH, "comet_results_3mt_FINETUNED_QE.csv")

# ----- Config -----
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda" and torch.cuda.is_bf16_supported():
    DTYPE = torch.bfloat16
elif DEVICE == "cuda":
    DTYPE = torch.float16
else:
    DTYPE = torch.float32

MAX_NEW_TOKENS = 96
QE_CKPT = "Unbabel/wmt20-comet-qe-da"
DA_CKPT = "Unbabel/wmt22-comet-da"

# ----- Load data -----
assert Path(CSV_PATH).exists(), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

required = {"src", "true_meaning"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"CSV must have columns {sorted(required)}. Missing: {sorted(missing)}")

if "literal_translation" not in df.columns:
    df["literal_translation"] = ""

for col in ["src", "true_meaning", "literal_translation"]:
    df[col] = df[col].fillna("").astype(str)
df = df[df["src"].str.strip().str.len() > 0].reset_index(drop=True)
print(f"[Info] Loaded {len(df)} rows from {CSV_PATH}")

# ----- Load model/tokenizer -----
print(f"[Info] Loading tokenizer from: {TOK_PATH}")
tok = AutoTokenizer.from_pretrained(TOK_PATH, trust_remote_code=True, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
EOS_ID = tok.eos_token_id

print(f"[Info] Loading fine-tuned model: {MODEL_PATH} on {DEVICE}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, device_map="auto", torch_dtype=DTYPE, trust_remote_code=True
).eval()

# ----- Few-shot prompt (same as above) -----
FEW_SHOTS = [
    ("उसकी बातें मेरे सर के ऊपर से गईं।", "I couldn’t make heads or tails of it."),
    ("वह टस से मस नहीं हुआ।",            "He wouldn’t budge an inch."),
    ("तेल देखो और तेल की धार देखो।",     "Let’s wait and see how it plays out."),
]
SYSTEM_MSG = (
    "You are an expert Hindi→English translator for IDIOMS. "
    "Convey the FIGURATIVE meaning; avoid literal renderings."
)
USER_INSTR = (
    "Task: Translate the Hindi idiom into natural English conveying the figurative sense.\n"
    "Rules:\n"
    "1) Avoid literal/word-for-word translations.\n"
    "2) Output ONLY the English translation (no labels, no quotes).\n"
    "3) Prefer common idiomatic English; otherwise paraphrase plainly.\n"
    "4) ≤ 20 words; natural everyday tone."
)

def build_prompt(tok, src_text: str, k_shots: int = 2) -> str:
    shots = random.sample(FEW_SHOTS, k=min(k_shots, len(FEW_SHOTS)))
    messages = [{"role": "system", "content": SYSTEM_MSG}]
    for hi, en in shots:
        messages.append({"role": "user", "content": f"{USER_INSTR}\n\nHindi: {hi}\nEnglish:"})
        messages.append({"role": "assistant", "content": en})
    messages.append({"role": "user", "content": f"{USER_INSTR}\n\nHindi: {src_text}\nEnglish:"})
    return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

_EN_LINE = re.compile(r"^[^\n\r]*", flags=re.IGNORECASE)
def clean_english(text: str) -> str:
    line = _EN_LINE.findall(text.strip())[0] if text.strip() else ""
    if line.lower().startswith("english:"): line = line[len("english:"):].strip()
    return re.sub(r'\s+', ' ', line.strip().strip('"').strip("'").strip("`")).strip()

def _prompt(text: str) -> str:
    return build_prompt(tok, text, k_shots=2)

@torch.no_grad()
def translate_once(text: str, *, do_sample: bool, temperature: float = 1.0, top_p: float = 0.9):
    enc = tok(_prompt(text), return_tensors="pt").to(next(model.parameters()).device)
    input_len = enc.input_ids.shape[1]
    gen_kwargs = dict(max_new_tokens=MAX_NEW_TOKENS, pad_token_id=tok.pad_token_id, eos_token_id=EOS_ID)
    if do_sample:
        gen_kwargs.update(dict(do_sample=True, temperature=float(temperature), top_p=float(top_p)))
    else:
        gen_kwargs.update(dict(do_sample=False))
    out = model.generate(**enc, **gen_kwargs)
    cont = tok.decode(out[0, input_len:], skip_special_tokens=True)
    return clean_english(cont)

def translate_3_variants(text: str):
    return [
        translate_once(text, do_sample=False),
        translate_once(text, do_sample=True,  temperature=0.7, top_p=0.9),
        translate_once(text, do_sample=True,  temperature=1.0, top_p=0.92),
    ]

# ----- Load COMET models (QE + DA) -----
print("[Info] Loading COMET QE …")
qe_model = load_from_checkpoint(download_model(QE_CKPT)).eval()
print("[Info] Loading COMET DA …")
da_model = load_from_checkpoint(download_model(DA_CKPT)).eval()

# ----- Prepare output columns -----
for c in [
    "mt_1","mt_2","mt_3",
    "QE_mt_1","QE_mt_2","QE_mt_3","QE_true","QE_literal","QE_dataset_mt",
    "DA_mt_1","DA_mt_2","DA_mt_3","DA_true","DA_literal",
    "mt_best","QE_mt_best","mt_best_from",
    "mt_best_DA_from","DA_mt_best",
    "literal_gt_ref"
]:
    df[c] = float("nan") if c.startswith(("QE_","DA_")) else ""

# ----- Generate & QE -----
qe_items, qe_locs = [], []
for i, row in df.iterrows():
    src = row["src"]
    mts = translate_3_variants(src)
    df.at[i, "mt_1"], df.at[i, "mt_2"], df.at[i, "mt_3"] = mts
    for j, mt in enumerate(mts, start=1):
        if isinstance(mt, str) and mt.strip():
            qe_items.append({"src": src, "mt": mt})
            qe_locs.append((i, f"QE_mt_{j}"))
    if row["true_meaning"].strip():
        qe_items.append({"src": src, "mt": row["true_meaning"]}); qe_locs.append((i, "QE_true"))
    if row["literal_translation"].strip():
        qe_items.append({"src": src, "mt": row["literal_translation"]}); qe_locs.append((i, "QE_literal"))

with torch.no_grad():
    qe_scores = qe_model.predict(qe_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), score in zip(qe_locs, qe_scores):
    df.at[i, field] = float(score)

# ----- Pick QE-best -----
def _norm(v):
    try: v = float(v); return v if not math.isnan(v) else -math.inf
    except: return -math.inf

best_cols, best_texts, best_qes = [], [], []
for _, r in df.iterrows():
    cands = [("mt_1", r["mt_1"], r["QE_mt_1"]),
             ("mt_2", r["mt_2"], r["QE_mt_2"]),
             ("mt_3", r["mt_3"], r["QE_mt_3"])]
    col, txt, qe = max(cands, key=lambda x: _norm(x[2]))
    best_cols.append(col); best_texts.append(txt); best_qes.append(qe)

df["mt_best"]      = best_texts
df["QE_mt_best"]   = best_qes
df["mt_best_from"] = best_cols

# ----- COMET-DA (ref = true_meaning) -----
da_items, da_locs = [], []
for i, r in df.iterrows():
    ref = r["true_meaning"]
    if not isinstance(ref, str) or not ref.strip(): continue
    for label in ["mt_1","mt_2","mt_3","literal_translation","true_meaning"]:
        hyp = r[label] if label in r else r.get(label, "")
        if isinstance(hyp, str) and hyp.strip():
            da_items.append({"src": r["src"], "mt": hyp, "ref": ref})
            field = f"DA_{'literal' if label=='literal_translation' else ('true' if label=='true_meaning' else label)}"
            da_locs.append((i, field))

with torch.no_grad():
    da_scores = da_model.predict(da_items, batch_size=32, progress_bar=False)["scores"]
for (i, field), s in zip(da_locs, da_scores):
    df.at[i, field] = float(s)

# Best-of-3 by DA among MT candidates
da_best_labels, da_best_scores = [], []
for _, r in df.iterrows():
    cands = [("mt_1", r.get("DA_mt_1", float("nan"))),
             ("mt_2", r.get("DA_mt_2", float("nan"))),
             ("mt_3", r.get("DA_mt_3", float("nan")))]
    cands = [(k, (v if isinstance(v,(int,float)) and not math.isnan(v) else -math.inf)) for k, v in cands]
    best_k, best_s = max(cands, key=lambda x: x[1])
    da_best_labels.append(best_k); da_best_scores.append(best_s if best_s!=-math.inf else float("nan"))

df["mt_best_DA_from"] = da_best_labels
df["DA_mt_best"]      = da_best_scores

# ----- Summaries -----
def _nanmean(s):
    vals = pd.to_numeric(s, errors="coerce").dropna().tolist()
    return sum(vals)/len(vals) if vals else float("nan")

df["literal_gt_ref"] = (
    pd.to_numeric(df["QE_literal"], errors="coerce") >
    pd.to_numeric(df["QE_true"],    errors="coerce")
)

total_rows       = len(df)
avg_qe_best_mt   = _nanmean(df["QE_mt_best"])
avg_qe_true      = _nanmean(df["QE_true"])
avg_qe_literal   = _nanmean(df["QE_literal"])
num_best_gt_ref  = int((pd.to_numeric(df["QE_mt_best"], errors="coerce") >
                         pd.to_numeric(df["QE_true"],    errors="coerce")).sum())
num_lit_gt_ref   = int(df["literal_gt_ref"].fillna(False).sum())

avg_da_best_mt   = _nanmean(df["DA_mt_best"])
avg_da_true      = _nanmean(df["DA_true"])
avg_da_literal   = _nanmean(df["DA_literal"])

print("\n=== SUMMARY (COMET-QE) — Llama COMET-QE finetune ===")
print(f"Samples                              : {total_rows}")
print(f"Average QE (best of 3 MT)            : {avg_qe_best_mt:.4f}")
print(f"Average QE (literal translation)     : {avg_qe_literal:.4f}")
print(f"Average QE (reference true_meaning)  : {avg_qe_true:.4f}")
print(f"Best-MT QE > true QE                 : {num_best_gt_ref} / {total_rows}")
print(f"Literal QE > true QE                 : {num_lit_gt_ref} / {total_rows}")

print("\n=== SUMMARY (COMET-DA; ref = true_meaning) — Llama COMET-QE finetune ===")
print(f"Average DA (best of 3 MT)            : {avg_da_best_mt:.4f}")
print(f"Average DA (true vs ref)             : {avg_da_true:.4f}")
print(f"Average DA (literal vs ref)          : {avg_da_literal:.4f}")

# ----- Save outputs -----
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print(f"\nSaved CSV : {OUT_CSV}")
print(f"Saved JSON: {OUT_JSON}")


AssertionError: CSV not found: /content/drive/MyDrive/GRPO-Hindi-Eng/llama3.2-3b-grpo-comet-qe/splits/test.csv